In [1]:
# Cell 1: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance
import glob
import warnings
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
warnings.filterwarnings('ignore')

# Define paths based on your repository structure
weather_data_dir = "weather_data"
stock_data_dir = "clean_data"

## 1. Loading and Initial Data Preparation

load both stock and weather data, then start preparing them for analysis.

In [2]:
# Cell 2: Load and prepare stock data from individual ticker files
print("Loading stock data from individual ticker files...")
# Get all CSV files in the clean_data directory
stock_files = glob.glob(os.path.join(stock_data_dir, "*.csv"))
print(f"Found {len(stock_files)} stock files")

# Load each file and add the ticker information
dataframes = []
for file in stock_files:
    ticker = os.path.basename(file).split('.')[0]  # Extract ticker from filename
    df = pd.read_csv(file)
    if 'Ticker' not in df.columns:  # Only add if not already present
        df['Ticker'] = ticker
    dataframes.append(df)

# Combine all stock data
stock_data = pd.concat(dataframes, ignore_index=True)
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

# Display basic information
print(f"Combined stock data shape: {stock_data.shape}")
print(f"Date range: {stock_data['Date'].min()} to {stock_data['Date'].max()}")
print(f"Number of unique stocks: {stock_data['Ticker'].nunique()}")
print("Example stock features:", list(stock_data.columns[:10]))
stock_data.head()

Loading stock data from individual ticker files...
Found 172 stock files
Combined stock data shape: (617824, 30)
Date range: 2010-01-04 00:00:00 to 2024-12-20 00:00:00
Number of unique stocks: 172
Example stock features: ['Date', 'Symbol', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume', 'ema15', 'ema50']


,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,ema15,ema50,...,vwap,roc_10,Target,Clopen,HighLow,log_price,Log5,Log15,Log30,Ticker
0,2013-01-02,ABBV,21.629181,35.119999,35.400002,34.099998,34.919998,13767900.0,NaN,NaN,...,NaN,NaN,0,1.005727,1.038123,3.558771,NaN,NaN,NaN,ABBV
1,2013-01-03,ABBV,21.450581,34.830002,35.000000,34.160000,35.000000,16739300.0,NaN,NaN,...,NaN,NaN,0,0.995143,1.024590,3.550479,NaN,NaN,NaN,ABBV
2,2013-01-04,ABBV,21.179594,34.389999,34.889999,34.250000,34.619999,21372100.0,NaN,NaN,...,NaN,NaN,1,0.993356,1.018686,3.537766,NaN,NaN,NaN,ABBV
3,2013-01-07,ABBV,21.222700,34.459999,35.450001,34.150002,34.150002,17897100.0,NaN,NaN,...,NaN,NaN,0,1.009078,1.038067,3.539799,NaN,NaN,NaN,ABBV
4,2013-01-08,ABBV,20.760809,33.709999,34.639999,33.360001,34.290001,17863300.0,NaN,NaN,...,NaN,NaN,1,0.983085,1.038369,3.517795,NaN,NaN,NaN,ABBV


In [3]:
# Cell 3: Load weather data
print("Loading weather data...")
daily_weather_path = os.path.join(weather_data_dir, "combined_daily_weather_data.csv")
weather_df = pd.read_csv(daily_weather_path)
weather_df['date'] = pd.to_datetime(weather_df['date'])

# For S&P 500 stocks, we'll primarily use New York weather
# Filter to keep only New York weather
print("Processing New York weather data...")
ny_weather = weather_df[weather_df['location'] == 'New_York'].copy()
ny_weather = ny_weather.rename(columns={'date': 'Date'})

# Select relevant weather features
weather_features = [
    'temperature_2m_mean', 'temperature_2m_max', 'temperature_2m_min',
    'wind_speed_10m_max', 'wind_gusts_10m_max', 'rain_sum',
    'snowfall_sum', 'precipitation_sum', 'precipitation_hours',
    'sunshine_duration', 'daylight_duration'
]
ny_weather = ny_weather[['Date'] + weather_features]

# Display basic information
print(f"NY weather data shape: {ny_weather.shape}")
print(f"Weather date range: {ny_weather['Date'].min()} to {ny_weather['Date'].max()}")
ny_weather.head()

Loading weather data...
Processing New York weather data...
NY weather data shape: (9133, 12)
Weather date range: 2000-02-02 04:00:00+00:00 to 2025-02-02 04:00:00+00:00


,Date,temperature_2m_mean,temperature_2m_max,temperature_2m_min,wind_speed_10m_max,wind_gusts_10m_max,rain_sum,snowfall_sum,precipitation_sum,precipitation_hours,sunshine_duration,daylight_duration
0,2000-02-02 04:00:00+00:00,-6.195917,-3.548,-8.747999,27.971327,52.199997,0.0,0.00,0.0,0.0,33045.746,36505.156
1,2000-02-03 04:00:00+00:00,-6.373001,-1.748,-11.297999,13.104197,28.800000,0.0,0.63,0.9,5.0,7588.906,36639.310
2,2000-02-04 04:00:00+00:00,-3.191750,0.002,-6.648000,9.178235,20.519999,0.0,0.56,0.6,2.0,29841.834,36774.555
3,2000-02-05 04:00:00+00:00,-3.554250,-0.648,-6.098000,18.193361,32.760002,0.0,0.14,0.0,0.0,33403.030,36910.740
4,2000-02-06 04:00:00+00:00,-4.579250,-0.248,-7.698000,18.720000,35.280000,0.0,0.00,0.0,0.0,33629.734,37047.720


In [4]:
# Cell 4: Handle Seasonality in Weather Data
print("Handling seasonality in weather data...")

# Add explicit seasonal features
ny_weather['month'] = ny_weather['Date'].dt.month
ny_weather['day_of_year'] = ny_weather['Date'].dt.dayofyear
ny_weather['season'] = np.ceil(ny_weather['month']/3).astype(int)
ny_weather['season'] = ny_weather['season'].replace({5:1})  # Fix any values over 4

# Map seasons to names for easier interpretation
season_map = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
ny_weather['season_name'] = ny_weather['season'].map(season_map)

# Create seasonal averages for temperature and precipitation
for feature in ['temperature_2m_mean', 'precipitation_sum']:
    # Calculate multi-year seasonal average for each day of year
    seasonal_avg = ny_weather.groupby('day_of_year')[feature].transform('mean')
    ny_weather[f'{feature}_seasonal_avg'] = seasonal_avg
    
    # Calculate deviation from seasonal norm
    ny_weather[f'{feature}_seasonal_deviation'] = ny_weather[feature] - ny_weather[f'{feature}_seasonal_avg']

# Handle specific seasonal anomalies
# Create seasonal volatility measure (how unusual is current weather for this time of year?)
for feature in ['temperature_2m_mean', 'precipitation_sum']:
    # Calculate multi-year seasonal standard deviation for each day of year
    seasonal_std = ny_weather.groupby('day_of_year')[feature].transform('std')
    ny_weather[f'{feature}_seasonal_volatility'] = seasonal_std
    
    # Calculate z-score compared to seasonal norm (how many standard deviations from normal?)
    ny_weather[f'{feature}_seasonal_zscore'] = (
        (ny_weather[feature] - ny_weather[f'{feature}_seasonal_avg']) / 
        ny_weather[f'{feature}_seasonal_volatility']
    ).fillna(0)  # Handle division by zero

# Identify extreme seasonal anomalies (more than 2 standard deviations from seasonal norm)
ny_weather['extreme_temp_anomaly'] = (
    (ny_weather['temperature_2m_mean_seasonal_zscore'].abs() > 2)
).astype(int)

ny_weather['extreme_precip_anomaly'] = (
    (ny_weather['precipitation_sum_seasonal_zscore'].abs() > 2)
).astype(int)

# Show examples of some extreme seasonal anomalies
extreme_examples = ny_weather[ny_weather['extreme_temp_anomaly'] == 1].sample(min(5, sum(ny_weather['extreme_temp_anomaly'])))
print("Examples of extreme temperature anomalies:")
print(extreme_examples[['Date', 'temperature_2m_mean', 'temperature_2m_mean_seasonal_avg', 
                 'temperature_2m_mean_seasonal_zscore', 'season_name']])

Handling seasonality in weather data...
Examples of extreme temperature anomalies:
                          Date  temperature_2m_mean  \
6233 2017-02-25 04:00:00+00:00            11.286835   
7535 2020-09-19 04:00:00+00:00            12.566001   
335  2001-01-02 04:00:00+00:00           -10.312583   
5497 2015-02-20 04:00:00+00:00           -15.454250   
7251 2019-12-10 04:00:00+00:00            10.805584   

      temperature_2m_mean_seasonal_avg  temperature_2m_mean_seasonal_zscore  \
6233                          1.113397                             2.146721   
7535                         18.431981                            -2.170017   
335                           0.373457                            -2.401723   
5497                          0.068397                            -2.887905   
7251                          2.258647                             2.024786   

     season_name  
6233      Winter  
7535      Summer  
335       Winter  
5497      Winter  
7251        Fall

In [5]:
# Cell 5: Engineer additional weather features
print("Engineering weather features...")
# Temperature features
ny_weather['temp_range'] = ny_weather['temperature_2m_max'] - ny_weather['temperature_2m_min']
ny_weather['extreme_heat'] = (ny_weather['temperature_2m_max'] > 30).astype(int)
ny_weather['extreme_cold'] = (ny_weather['temperature_2m_min'] < 0).astype(int)

# Precipitation features
ny_weather['heavy_rain'] = (ny_weather['rain_sum'] > 10).astype(int)
ny_weather['snow_day'] = (ny_weather['snowfall_sum'] > 0).astype(int)
ny_weather['any_precipitation'] = (ny_weather['precipitation_sum'] > 0).astype(int)

# Wind features
ny_weather['strong_wind'] = (ny_weather['wind_gusts_10m_max'] > 40).astype(int)

# Sunshine and daylight features
ny_weather['sunshine_ratio'] = ny_weather['sunshine_duration'] / ny_weather['daylight_duration']
ny_weather['sunshine_ratio'] = ny_weather['sunshine_ratio'].fillna(0)  # Handle division by zero

# Add lag features (previous day's weather)
for feature in ['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max']:
    ny_weather[f'{feature}_lag1'] = ny_weather[feature].shift(1)
    ny_weather[f'{feature}_lag2'] = ny_weather[feature].shift(2)
    ny_weather[f'{feature}_lag3'] = ny_weather[feature].shift(3)

# Add rolling statistics for temperature patterns
for window in [3, 7]:
    ny_weather[f'temp_mean_rolling{window}'] = ny_weather['temperature_2m_mean'].rolling(window=window).mean()
    ny_weather[f'temp_std_rolling{window}'] = ny_weather['temperature_2m_mean'].rolling(window=window).std()

# Calculate weather changes
ny_weather['temp_change'] = ny_weather['temperature_2m_mean'].diff()
ny_weather['rain_change'] = ny_weather['rain_sum'].diff()

# Fill NaN values created by shifts and rolling calculations
ny_weather = ny_weather.fillna(method='bfill').fillna(method='ffill')

# Display the newly created features
print("Weather features created. Total features:", len(ny_weather.columns))
new_weather_features = [col for col in ny_weather.columns if col not in ['Date'] + weather_features]
print("New weather features examples:", new_weather_features[:5])

Engineering weather features...
Weather features created. Total features: 49
New weather features examples: ['month', 'day_of_year', 'season', 'season_name', 'temperature_2m_mean_seasonal_avg']


## Handling Seasonality
adding seasoal features to capture known market seasonality

In [6]:
# Cell 6: Handle Seasonality in Stock Data
print("Adding seasonal features to stock data...")
stock_data['month'] = stock_data['Date'].dt.month
stock_data['day_of_year'] = stock_data['Date'].dt.dayofyear
stock_data['season'] = np.ceil(stock_data['month']/3).astype(int)
stock_data['season'] = stock_data['season'].replace({5:1})  # Fix any values over 4
stock_data['quarter'] = stock_data['Date'].dt.quarter
stock_data['year'] = stock_data['Date'].dt.year

# Standard seasonal stock market effects
stock_data['january_effect'] = (stock_data['month'] == 1).astype(int)
stock_data['december_effect'] = (stock_data['month'] == 12).astype(int)
stock_data['october_effect'] = (stock_data['month'] == 10).astype(int)
stock_data['summer_doldrums'] = ((stock_data['month'] >= 6) & (stock_data['month'] <= 8)).astype(int)
stock_data['quarter_end'] = stock_data['month'].isin([3, 6, 9, 12]).astype(int)
stock_data['day_of_week'] = stock_data['Date'].dt.dayofweek
stock_data['is_monday'] = (stock_data['day_of_week'] == 0).astype(int)
stock_data['is_friday'] = (stock_data['day_of_week'] == 4).astype(int)

# Display some samples to verify
print("Stock data with seasonal features:")
stock_data[['Date', 'month', 'season', 'quarter', 'january_effect', 'summer_doldrums']].head()

Adding seasonal features to stock data...
Stock data with seasonal features:


,Date,month,season,quarter,january_effect,summer_doldrums
0,2013-01-02,1,1,1,1,0
1,2013-01-03,1,1,1,1,0
2,2013-01-04,1,1,1,1,0
3,2013-01-07,1,1,1,1,0
4,2013-01-08,1,1,1,1,0


In [7]:
# Cell 7: Merge stock data with weather data
print("Merging stock and weather data...")

# 1. Convert dates to date-only format by extracting just the date part (year, month, day)
print("Converting dates to date-only format...")
stock_data['Date_Only'] = pd.to_datetime(stock_data['Date']).dt.date
ny_weather['Date_Only'] = pd.to_datetime(ny_weather['Date']).dt.date

# 2. Check date ranges to understand overlap
print(f"Stock data date range: {min(stock_data['Date_Only'])} to {max(stock_data['Date_Only'])}")
print(f"Weather data date range: {min(ny_weather['Date_Only'])} to {max(ny_weather['Date_Only'])}")

# 3. Find overlapping dates
stock_dates = set(stock_data['Date_Only'])
weather_dates = set(ny_weather['Date_Only'])
overlapping_dates = stock_dates.intersection(weather_dates)
print(f"Number of overlapping dates: {len(overlapping_dates)}")

# 4. Filter both datasets to only use overlapping dates
stock_data_filtered = stock_data[stock_data['Date_Only'].isin(overlapping_dates)]
weather_data_filtered = ny_weather[ny_weather['Date_Only'].isin(overlapping_dates)]

print(f"Filtered stock data shape: {stock_data_filtered.shape}")
print(f"Filtered weather data shape: {weather_data_filtered.shape}")

# 5. For weather data with multiple observations per day, take the daily average
weather_daily = weather_data_filtered.groupby('Date_Only').mean(numeric_only=True).reset_index()

# Also preserve non-numeric columns that we want to keep
for col in ['season', 'season_name']:
    if col in weather_data_filtered.columns:
        # Take the most common value for categorical data
        weather_daily[col] = weather_data_filtered.groupby('Date_Only')[col].first().values

print(f"Weather data after aggregating to daily: {weather_daily.shape}")

# 6. Merge using the date-only column
combined_df = pd.merge(stock_data_filtered, weather_daily, on='Date_Only', suffixes=('', '_weather'))
print(f"Combined data shape: {combined_df.shape}")

# 7. Add seasonal features if not present
if 'season' not in combined_df.columns:
    print("Adding season information to combined data...")
    combined_df['month'] = combined_df['Date'].dt.month
    combined_df['season'] = np.ceil(combined_df['month']/3).astype(int)
    combined_df['season'] = combined_df['season'].replace({5:1})  # Fix any values over 4
    
    # Map seasons to names for easier interpretation
    season_map = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
    combined_df['season_name'] = combined_df['season'].map(season_map)

# 8. Basic statistics on the merged dataset
if not combined_df.empty:
    print(f"Number of unique dates: {combined_df['Date_Only'].nunique()}")
    print(f"Date range of combined data: {combined_df['Date_Only'].min()} to {combined_df['Date_Only'].max()}")
    print(f"Number of unique stocks: {combined_df['Ticker'].nunique()}")
    print(f"Number of seasons represented: {combined_df['season'].nunique()}")

    # 9. Show sample results
    print("\nSample of combined data:")
    display(combined_df.head(3))
else:
    print("Combined dataframe is empty. Check date ranges for possible issues.")

Merging stock and weather data...
Converting dates to date-only format...
Stock data date range: 2010-01-04 to 2024-12-20
Weather data date range: 2000-02-02 to 2025-02-02
Number of overlapping dates: 3768
Filtered stock data shape: (617824, 44)
Filtered weather data shape: (3768, 50)
Weather data after aggregating to daily: (3768, 49)
Combined data shape: (617824, 92)
Number of unique dates: 3768
Date range of combined data: 2010-01-04 to 2024-12-20
Number of unique stocks: 172
Number of seasons represented: 4

Sample of combined data:


,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,ema15,ema50,...,wind_speed_10m_max_lag1,wind_speed_10m_max_lag2,wind_speed_10m_max_lag3,temp_mean_rolling3,temp_std_rolling3,temp_mean_rolling7,temp_std_rolling7,temp_change,rain_change,season_name
0,2013-01-02,ABBV,21.629181,35.119999,35.400002,34.099998,34.919998,13767900.0,NaN,NaN,...,19.134260,18.204042,29.515608,-1.792444,2.626080,-0.904250,2.289725,-5.243750,0.0,Winter
1,2013-01-03,ABBV,21.450581,34.830002,35.000000,34.160000,35.000000,16739300.0,NaN,NaN,...,16.263872,19.134260,18.204042,-2.748000,3.023878,-1.938476,1.976605,0.012499,0.0,Winter
2,2013-01-04,ABBV,21.179594,34.389999,34.889999,34.250000,34.619999,21372100.0,NaN,NaN,...,14.039999,16.263872,19.134260,-3.500778,1.720034,-2.025679,1.936430,2.972917,0.0,Winter


In [8]:
# Cell 8: Create interaction features between weather and stock seasonality
print("Creating weather-seasonality interaction features...")

# Verify that required columns exist before proceeding
required_columns = ['season', 'temperature_2m_mean', 'precipitation_sum']
missing_columns = [col for col in required_columns if col not in combined_df.columns]

if missing_columns:
    print(f"Warning: Missing required columns: {missing_columns}")
    print("Available columns:", combined_df.columns.tolist())
    print("Skipping interaction feature creation.")
else:
    # For each season, create season-specific weather anomaly features
    for season in range(1, 5):
        season_data = combined_df[combined_df['season'] == season]
        if len(season_data) > 0:
            print(f"Creating features for season {season} with {len(season_data)} rows")
            # Calculate season-specific weather anomalies
            for feature in ['temperature_2m_mean', 'precipitation_sum']:
                season_avg = season_data[feature].mean()
                season_std = season_data[feature].std()
                # Handle case where std is 0 to avoid division by zero
                if season_std > 0:
                    combined_df.loc[combined_df['season'] == season, f'{feature}_season{season}_zscore'] = (
                        (combined_df.loc[combined_df['season'] == season, feature] - season_avg) / season_std
                    ).fillna(0)
                else:
                    combined_df.loc[combined_df['season'] == season, f'{feature}_season{season}_zscore'] = 0
    
    # Create industry-specific seasonal features if 'Sector' column exists
    if 'Sector' in combined_df.columns:
        print("Creating sector-specific seasonal features...")
        # Create sector-season interaction features
        for season in range(1, 5):
            combined_df[f'season_{season}_flag'] = (combined_df['season'] == season).astype(int)
            
        # For each sector, create sector-season interaction terms
        sectors = combined_df['Sector'].unique()
        for sector in sectors:
            sector_mask = (combined_df['Sector'] == sector).astype(int)
            for season in range(1, 5):
                season_mask = (combined_df['season'] == season).astype(int)
                combined_df[f'{sector}_season_{season}'] = sector_mask * season_mask
    
    print("Interaction features created.")
    print(f"Combined data now has {combined_df.shape[1]} columns")

Creating weather-seasonality interaction features...
Creating features for season 1 with 150244 rows
Creating features for season 2 with 155029 rows
Creating features for season 3 with 156982 rows
Creating features for season 4 with 155569 rows
Interaction features created.
Combined data now has 100 columns


In [9]:
# Cell 9: Define feature sets for comparison
print("Preparing feature sets...")
# Exclude these columns from features
exclude_cols = ['Date', 'Date_Only', 'Ticker', 'Symbol', 'Target', 'location', 'season_name']

# Also explicitly exclude any datetime or date columns
date_cols = combined_df.select_dtypes(include=['datetime64', 'datetime64[ns]']).columns.tolist()
# Add any other columns that might be problematic
obj_cols = combined_df.select_dtypes(include=['object']).columns.tolist()

# Add all excluded columns together
all_exclude_cols = exclude_cols + date_cols + obj_cols

# Get stock features (including stock seasonality features)
base_features = [col for col in stock_data.columns 
                 if col not in all_exclude_cols 
                 and col in combined_df.columns  # Make sure feature exists in combined dataframe
                 and pd.api.types.is_numeric_dtype(combined_df[col])]  # Only include numeric columns

# Get weather-only features (including weather seasonality features)
weather_only_features = [col for col in combined_df.columns 
                        if col not in stock_data.columns 
                        and col not in all_exclude_cols
                        and pd.api.types.is_numeric_dtype(combined_df[col])]  # Only include numeric columns

# Get all features
all_features = base_features + weather_only_features

# Print feature counts
print(f"Number of stock features: {len(base_features)}")
print(f"Number of weather features: {len(weather_only_features)}")
print(f"Total number of features: {len(all_features)}")

# Prepare data for modeling
print("Preparing data for modeling...")
# Check target variable
print("Target value counts:")
print(combined_df['Target'].value_counts())

# Make sure the target is properly encoded as 0/1
combined_df['Target'] = combined_df['Target'].astype(int)

y = combined_df['Target']
X_base = combined_df[base_features]
X_all = combined_df[all_features]

# Verify that we only have numeric data
print(f"X_base data types: {X_base.dtypes.value_counts()}")
print(f"X_all data types: {X_all.dtypes.value_counts()}")

# Display first few features of each type
print("\nSample stock features:", base_features[:5])
print("\nSample weather features:", weather_only_features[:5])

Preparing feature sets...
Number of stock features: 39
Number of weather features: 55
Total number of features: 94
Preparing data for modeling...
Target value counts:
Target
1    321555
0    296269
Name: count, dtype: int64
X_base data types: float64    26
int32      13
Name: count, dtype: int64
X_all data types: float64    80
int32      14
Name: count, dtype: int64

Sample stock features: ['Adj Close', 'Close', 'High', 'Low', 'Open']

Sample weather features: ['temperature_2m_mean', 'temperature_2m_max', 'temperature_2m_min', 'wind_speed_10m_max', 'wind_gusts_10m_max']


In [10]:
# Cell 10: Updated function to evaluate models with different feature sets
def evaluate_model(X, y, feature_set_name):
    print(f"Input data shapes: X={X.shape}, y={y.shape}")
    
    # Make sure we only include numeric features
    numeric_columns = X.select_dtypes(include=['number']).columns
    if len(numeric_columns) < X.shape[1]:
        print(f"Warning: Dropping {X.shape[1] - len(numeric_columns)} non-numeric columns")
        X = X[numeric_columns]
    
    # Check target values
    print("Target value counts:")
    print(y.value_counts())
    
    # Convert target to int
    y = y.astype(int)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train a Random Forest (better than Decision Tree for this task)
    rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=20,
        min_samples_leaf=4,
        min_samples_split=10,
        random_state=42,
        n_jobs=-1  # Use all available cores
    )
    
    print(f"Training {feature_set_name} model...")
    rf.fit(X_train_scaled, y_train)
    
    # Evaluate
    y_pred = rf.predict(X_test_scaled)
    y_pred_proba = rf.predict_proba(X_test_scaled)[:, 1]  # Probability of class 1
    
    # Basic metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Advanced metrics
    from sklearn.metrics import roc_auc_score, average_precision_score, log_loss, brier_score_loss
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    pr_auc = average_precision_score(y_test, y_pred_proba)
    log_loss_score = log_loss(y_test, y_pred_proba)
    brier = brier_score_loss(y_test, y_pred_proba)
    
    # Calculate feature importance using the model's built-in importance
    feature_imp = pd.DataFrame({
        'Feature': X.columns,
        'Importance': rf.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print(f"\n--- {feature_set_name} Model Results ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}")
    print(f"PR-AUC: {pr_auc:.4f}")
    print(f"Log Loss: {log_loss_score:.4f}")
    print(f"Brier Score: {brier:.4f}")
    
    print("\nTop 10 Important Features:")
    print(feature_imp.head(10))
    
    # Calculate confusion matrix for better understanding
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Visualize confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Confusion Matrix - {feature_set_name}')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.tight_layout()
    plt.savefig(os.path.join(weather_data_dir, f'{feature_set_name.replace(" ", "_").lower()}_confusion_matrix.png'))
    plt.close()
    
    # Return model, metrics, and feature importance
    metrics_dict = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'log_loss': log_loss_score,
        'brier_score': brier
    }
    
    return rf, metrics_dict, feature_imp

In [11]:
# Cell 11: Evaluate base model (stock data only) - Modified to avoid CSV saving
print("\nEvaluating base model (stock data only)...")
base_model, base_metrics, base_importance = evaluate_model(X_base, y, "Base Model (Stock Data Only)")

# Visualize base model feature importance
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=base_importance.head(15))
plt.title('Top 15 Features (Stock Data Only Model)')
plt.tight_layout()
plt.savefig(os.path.join(weather_data_dir, 'base_feature_importance.png'))
plt.close()



Evaluating base model (stock data only)...
Input data shapes: X=(617824, 39), y=(617824,)
Target value counts:
Target
1    321555
0    296269
Name: count, dtype: int64
Training Base Model (Stock Data Only) model...

--- Base Model (Stock Data Only) Model Results ---
Accuracy: 0.6123
Precision: 0.5986
Recall: 0.7806
F1 Score: 0.6776
ROC-AUC: 0.6630
PR-AUC: 0.6732
Log Loss: 0.6644
Brier Score: 0.2360

Top 10 Important Features:
        Feature  Importance
27  day_of_year    0.088963
23         Log5    0.046158
20       Clopen    0.045375
19       roc_10    0.043077
21      HighLow    0.042909
24        Log15    0.042007
14      stoch_k    0.041432
15      stoch_d    0.041363
25        Log30    0.040607
13          rsi    0.040016


In [12]:
# Cell 12: Evaluate combined model (stock + weather) - Modified to avoid CSV saving
print("\nEvaluating combined model (stock + weather)...")
all_model, all_metrics, all_importance = evaluate_model(X_all, y, "Combined Model (Stock + Weather)")

# Visualize feature importance
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=all_importance.head(15))
plt.title('Top 15 Features (Stock + Weather Model)')
plt.tight_layout()
plt.savefig(os.path.join(weather_data_dir, 'feature_importance.png'))
plt.close()

# Weather features in the top features
weather_in_top = [feature for feature in all_importance.head(20)['Feature'] 
                 if feature in weather_only_features]
print("\nWeather features in top 20 important features:")
print(weather_in_top)

# Create ROC curve comparison
plt.figure(figsize=(10, 8))
from sklearn.metrics import roc_curve
# Function to plot ROC curves for both models
def plot_roc_comparison(X_base, X_all, y, base_model, all_model):
    # Get test data
    _, X_test_base, _, y_test = train_test_split(X_base, y, test_size=0.25, random_state=42)
    _, X_test_all, _, _ = train_test_split(X_all, y, test_size=0.25, random_state=42)
    
    # Scale test data
    scaler_base = StandardScaler()
    scaler_all = StandardScaler()
    X_test_base_scaled = scaler_base.fit_transform(X_test_base)
    X_test_all_scaled = scaler_all.fit_transform(X_test_all)
    
    # Get predictions
    y_pred_base = base_model.predict_proba(X_test_base_scaled)[:, 1]
    y_pred_all = all_model.predict_proba(X_test_all_scaled)[:, 1]
    
    # Calculate ROC
    fpr_base, tpr_base, _ = roc_curve(y_test, y_pred_base)
    fpr_all, tpr_all, _ = roc_curve(y_test, y_pred_all)
    
    # Plot
    plt.figure(figsize=(10, 8))
    plt.plot(fpr_base, tpr_base, label=f'Base Model (AUC = {base_metrics["roc_auc"]:.4f})')
    plt.plot(fpr_all, tpr_all, label=f'Combined Model (AUC = {all_metrics["roc_auc"]:.4f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve Comparison')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(os.path.join(weather_data_dir, 'roc_curve_comparison.png'))
    plt.close()

# Plot ROC curve comparison
plot_roc_comparison(X_base, X_all, y, base_model, all_model)


Evaluating combined model (stock + weather)...
Input data shapes: X=(617824, 94), y=(617824,)
Target value counts:
Target
1    321555
0    296269
Name: count, dtype: int64
Training Combined Model (Stock + Weather) model...

--- Combined Model (Stock + Weather) Model Results ---
Accuracy: 0.7076
Precision: 0.6927
Recall: 0.7902
F1 Score: 0.7383
ROC-AUC: 0.7790
PR-AUC: 0.7786
Log Loss: 0.5956
Brier Score: 0.2039

Top 10 Important Features:
                    Feature  Importance
42       wind_speed_10m_max    0.020321
78  wind_speed_10m_max_lag2    0.019868
77  wind_speed_10m_max_lag1    0.019764
84              temp_change    0.019116
83        temp_std_rolling7    0.019107
81        temp_std_rolling3    0.018891
23                     Log5    0.018555
49        daylight_duration    0.018501
79  wind_speed_10m_max_lag3    0.018464
20                   Clopen    0.018342

Weather features in top 20 important features:
['wind_speed_10m_max', 'wind_speed_10m_max_lag2', 'wind_speed_10m_max

<Figure size 1000x800 with 0 Axes>

In [13]:
# Cell 13: Evaluate seasonality-specific models - Modified to avoid CSV saving
print("\nEvaluating seasonality-specific models...")
seasons = combined_df['season'].unique()
season_results = {}

for season in seasons:
    season_name = season_map.get(season, f"Season {season}")
    print(f"\nEvaluating {season_name} model...")
    season_mask = combined_df['season'] == season
    
    if sum(season_mask) > 100:  # Only evaluate if enough data
        season_X = X_all[season_mask]
        season_y = y[season_mask]
        
        _, season_metrics, season_imp = evaluate_model(
            season_X, season_y, f"{season_name} Model"
        )
        
        season_results[season_name] = {
            'metrics': season_metrics,
            'top_features': season_imp.head(5)['Feature'].tolist()
        }
        
        # Visualize seasonal feature importance
        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=season_imp.head(10))
        plt.title(f'Top 10 Features - {season_name} Model')
        plt.tight_layout()
        plt.savefig(os.path.join(weather_data_dir, f'{season_name}_feature_importance.png'))
        plt.close()

# Display summary of seasonal results
print("\nSeasonal Model Summary:")
for season, results in season_results.items():
    print(f"\n{season}:")
    print(f"  Accuracy: {results['metrics']['accuracy']:.4f}")
    print(f"  ROC-AUC: {results['metrics']['roc_auc']:.4f}")
    print(f"  F1 Score: {results['metrics']['f1']:.4f}")
    print(f"  Top features: {results['top_features']}")

# Create comparison chart of seasonal metrics
seasonal_metrics_df = pd.DataFrame({
    'Season': [],
    'Metric': [],
    'Value': []
})

for season, results in season_results.items():
    for metric_name, metric_value in results['metrics'].items():
        seasonal_metrics_df = pd.concat([seasonal_metrics_df, pd.DataFrame({
            'Season': [season],
            'Metric': [metric_name],
            'Value': [metric_value]
        })], ignore_index=True)

# Visualize key seasonal metrics
plt.figure(figsize=(14, 10))
key_metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
key_metrics_df = seasonal_metrics_df[seasonal_metrics_df['Metric'].isin(key_metrics)]

sns.barplot(x='Season', y='Value', hue='Metric', data=key_metrics_df)
plt.title('Comparison of Key Metrics Across Seasons')
plt.ylim(0, 1)
plt.grid(True, alpha=0.3)
plt.savefig(os.path.join(weather_data_dir, 'seasonal_metrics_comparison.png'))
plt.close()


Evaluating seasonality-specific models...

Evaluating Winter model...
Input data shapes: X=(150244, 94), y=(150244,)
Target value counts:
Target
1    79323
0    70921
Name: count, dtype: int64
Training Winter Model model...

--- Winter Model Model Results ---
Accuracy: 0.7139
Precision: 0.7123
Recall: 0.7633
F1 Score: 0.7369
ROC-AUC: 0.7912
PR-AUC: 0.7996
Log Loss: 0.5491
Brier Score: 0.1858

Top 10 Important Features:
        Feature  Importance
25        Log30    0.024958
23         Log5    0.024717
21      HighLow    0.024226
24        Log15    0.023773
20       Clopen    0.023755
19       roc_10    0.023442
8          macd    0.023185
14      stoch_k    0.023081
13          rsi    0.022651
9   macd_signal    0.022378

Evaluating Spring model...
Input data shapes: X=(155029, 94), y=(155029,)
Target value counts:
Target
1    80371
0    74658
Name: count, dtype: int64
Training Spring Model model...

--- Spring Model Model Results ---
Accuracy: 0.7200
Precision: 0.7108
Recall: 0.7707


In [14]:
# Cell 14: Time series cross-validation - Modified to avoid CSV saving and optimize performance
print("\nPerforming time series cross-validation...")
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Define a smaller number of CV splits for faster processing
tscv = TimeSeriesSplit(n_splits=3)  # Reduced from 5 to 3
base_cv_scores = []
all_cv_scores = []
seasonal_cv_scores = {1: [], 2: [], 3: [], 4: []}

# Additional metrics for cross-validation
base_cv_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'roc_auc': []}
all_cv_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'roc_auc': []}

# Get a sample of the data if it's too large
sample_size = 50000  # Adjust based on your system capabilities
if len(X_all) > sample_size:
    # Use stratified sampling to maintain class balance
    from sklearn.model_selection import train_test_split
    _, X_all_sample, _, y_sample = train_test_split(
        X_all, y, test_size=sample_size/len(X_all), stratify=y, random_state=42
    )
    X_base_sample = X_base.loc[X_all_sample.index]
    combined_df_sample = combined_df.loc[X_all_sample.index]
    
    print(f"Using a sample of {sample_size} rows for cross-validation instead of {len(X_all)} total rows")
    X_all_cv = X_all_sample
    X_base_cv = X_base_sample
    y_cv = y_sample
    combined_df_cv = combined_df_sample
else:
    X_all_cv = X_all
    X_base_cv = X_base
    y_cv = y
    combined_df_cv = combined_df

print("Running cross-validation (this may take some time)...")

# Use lighter RandomForest for CV
rf_params = {
    'n_estimators': 50,       # Fewer trees
    'max_depth': 10,          # Lower depth
    'min_samples_leaf': 10,   # More aggressive pruning
    'min_samples_split': 20,  # More aggressive pruning
    'max_features': 'sqrt',   # Use sqrt of features for faster training
    'random_state': 42,
    'n_jobs': 4,              # Limit parallel jobs
    'verbose': 0              # No verbose output during CV
}

for fold, (train_idx, test_idx) in enumerate(tscv.split(X_all_cv)):
    print(f"\nProcessing fold {fold+1}/{tscv.n_splits}...")
    
    # Base model (stock only)
    X_train_base, X_test_base = X_base_cv.iloc[train_idx], X_base_cv.iloc[test_idx]
    y_train, y_test = y_cv.iloc[train_idx], y_cv.iloc[test_idx]
    
    # Get season info for test set
    test_seasons = combined_df_cv.iloc[test_idx]['season'].values
    
    # Train base model
    print("Training base model...")
    scaler_base = StandardScaler()
    X_train_base_scaled = scaler_base.fit_transform(X_train_base)
    X_test_base_scaled = scaler_base.transform(X_test_base)
    
    base_rf = RandomForestClassifier(**rf_params)
    base_rf.fit(X_train_base_scaled, y_train)
    base_preds = base_rf.predict(X_test_base_scaled)
    
    # Calculate basic metrics for base model
    base_acc = accuracy_score(y_test, base_preds)
    base_prec = precision_score(y_test, base_preds, zero_division=0)
    base_rec = recall_score(y_test, base_preds, zero_division=0)
    base_f1 = f1_score(y_test, base_preds, zero_division=0)
    
    # Get probabilities for ROC AUC - with error handling
    try:
        base_probs = base_rf.predict_proba(X_test_base_scaled)[:, 1]
        base_roc_auc = roc_auc_score(y_test, base_probs)
    except Exception as e:
        print(f"Warning: Could not calculate ROC AUC for base model: {e}")
        base_roc_auc = float('nan')
    
    # Store base model metrics
    base_cv_scores.append(base_acc)
    base_cv_metrics['accuracy'].append(base_acc)
    base_cv_metrics['precision'].append(base_prec)
    base_cv_metrics['recall'].append(base_rec)
    base_cv_metrics['f1'].append(base_f1)
    base_cv_metrics['roc_auc'].append(base_roc_auc)
    
    print(f"Base model - Accuracy: {base_acc:.4f}, F1: {base_f1:.4f}")
    
    # Combined model (stock + weather)
    print("Training combined model...")
    X_train_all, X_test_all = X_all_cv.iloc[train_idx], X_all_cv.iloc[test_idx]
    
    scaler_all = StandardScaler()
    X_train_all_scaled = scaler_all.fit_transform(X_train_all)
    X_test_all_scaled = scaler_all.transform(X_test_all)
    
    all_rf = RandomForestClassifier(**rf_params)
    all_rf.fit(X_train_all_scaled, y_train)
    all_preds = all_rf.predict(X_test_all_scaled)
    
    # Calculate metrics for combined model
    all_acc = accuracy_score(y_test, all_preds)
    all_prec = precision_score(y_test, all_preds, zero_division=0)
    all_rec = recall_score(y_test, all_preds, zero_division=0)
    all_f1 = f1_score(y_test, all_preds, zero_division=0)
    
    # Get probabilities for ROC AUC - with error handling
    try:
        all_probs = all_rf.predict_proba(X_test_all_scaled)[:, 1]
        all_roc_auc = roc_auc_score(y_test, all_probs)
    except Exception as e:
        print(f"Warning: Could not calculate ROC AUC for combined model: {e}")
        all_roc_auc = float('nan')
    
    # Store combined model metrics
    all_cv_scores.append(all_acc)
    all_cv_metrics['accuracy'].append(all_acc)
    all_cv_metrics['precision'].append(all_prec)
    all_cv_metrics['recall'].append(all_rec)
    all_cv_metrics['f1'].append(all_f1)
    all_cv_metrics['roc_auc'].append(all_roc_auc)
    
    print(f"Combined model - Accuracy: {all_acc:.4f}, F1: {all_f1:.4f}")
    
    # Calculate seasonal performance
    print("Analyzing seasonal performance...")
    for season in range(1, 5):
        season_mask = test_seasons == season
        if np.sum(season_mask) > 0:
            season_y_test = y_test.iloc[season_mask] if isinstance(y_test, pd.Series) else y_test[season_mask]
            season_base_preds = base_preds[season_mask]
            season_all_preds = all_preds[season_mask]
            
            # Compare performance improvement by season
            if len(season_y_test) > 0:
                try:
                    base_season_acc = accuracy_score(season_y_test, season_base_preds)
                    all_season_acc = accuracy_score(season_y_test, season_all_preds)
                    seasonal_cv_scores[season].append(all_season_acc - base_season_acc)
                    print(f"  Season {season}: Improvement: {all_season_acc - base_season_acc:.4f}")
                except Exception as e:
                    print(f"  Error calculating season {season} metrics: {e}")

print("\n--- Time Series Cross-Validation Results ---")
print(f"Base Model Avg Accuracy: {np.mean(base_cv_scores):.4f}")
print(f"Combined Model Avg Accuracy: {np.mean(all_cv_scores):.4f}")
print(f"Overall Improvement: {np.mean(all_cv_scores) - np.mean(base_cv_scores):.4f}")

# Create table of all CV metrics
try:
    cv_metrics_table = pd.DataFrame({
        'Metric': list(base_cv_metrics.keys()),
        'Base Model': [np.nanmean(base_cv_metrics[m]) for m in base_cv_metrics],
        'Combined Model': [np.nanmean(all_cv_metrics[m]) for m in all_cv_metrics],
        'Improvement': [np.nanmean(all_cv_metrics[m]) - np.nanmean(base_cv_metrics[m]) 
                    for m in base_cv_metrics]
    })
    
    # Calculate relative improvement with error handling for division by zero
    rel_imp = []
    for m in base_cv_metrics.keys():
        base_val = np.nanmean(base_cv_metrics[m])
        if base_val != 0:
            rel_imp.append(100 * (np.nanmean(all_cv_metrics[m]) - base_val) / base_val)
        else:
            rel_imp.append(float('nan'))
    
    cv_metrics_table['Relative Improvement (%)'] = rel_imp
    
    print("\n--- Detailed CV Metrics Comparison ---")
    print(cv_metrics_table)
except Exception as e:
    print(f"Error creating metrics table: {e}")

# Print seasonal improvements
print("\n--- Seasonal Improvement Analysis ---")
for season in range(1, 5):
    if seasonal_cv_scores[season]:
        avg_improvement = np.mean(seasonal_cv_scores[season])
        season_name = season_map.get(season, f"Season {season}")
        print(f"{season_name}: {avg_improvement:.4f} accuracy improvement")

# Visualize the results
try:
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(base_cv_scores)), base_cv_scores, width=0.4, label='Stock Only', alpha=0.7)
    plt.bar([x + 0.4 for x in range(len(all_cv_scores))], all_cv_scores, 
            width=0.4, label='Stock + Weather', alpha=0.7)
    plt.xlabel('Fold')
    plt.ylabel('Accuracy')
    plt.title('Cross-Validation Results: Stock vs. Stock+Weather Models')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(weather_data_dir, 'cv_results.png'))
    plt.close()

    # Visualize CV metrics comparison
    plt.figure(figsize=(12, 8))
    metrics_to_plot = [m for m in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc'] 
                      if not np.isnan(np.nanmean(base_cv_metrics[m]))]
    
    x = np.arange(len(metrics_to_plot))
    width = 0.35

    plt.bar(x - width/2, [np.nanmean(base_cv_metrics[m]) for m in metrics_to_plot], width, 
            label='Base Model')
    plt.bar(x + width/2, [np.nanmean(all_cv_metrics[m]) for m in metrics_to_plot], width, 
            label='Combined Model')

    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('Cross-Validation Metrics Comparison')
    plt.xticks(x, metrics_to_plot)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(weather_data_dir, 'cv_metrics_comparison.png'))
    plt.close()
except Exception as e:
    print(f"Error creating visualization: {e}")


Performing time series cross-validation...
Using a sample of 50000 rows for cross-validation instead of 617824 total rows
Running cross-validation (this may take some time)...

Processing fold 1/3...
Training base model...
Base model - Accuracy: 0.5096, F1: 0.6097
Training combined model...
Combined model - Accuracy: 0.5623, F1: 0.6354
Analyzing seasonal performance...
  Season 1: Improvement: 0.0527
  Season 2: Improvement: 0.0522
  Season 3: Improvement: 0.0469
  Season 4: Improvement: 0.0592

Processing fold 2/3...
Training base model...
Base model - Accuracy: 0.5207, F1: 0.6326
Training combined model...
Combined model - Accuracy: 0.5964, F1: 0.6785
Analyzing seasonal performance...
  Season 1: Improvement: 0.0611
  Season 2: Improvement: 0.0823
  Season 3: Improvement: 0.0910
  Season 4: Improvement: 0.0672

Processing fold 3/3...
Training base model...
Base model - Accuracy: 0.5235, F1: 0.6419
Training combined model...
Combined model - Accuracy: 0.6090, F1: 0.6904
Analyzing sea

In [15]:
# Cell 15: Stock-specific seasonal analysis - Modified to avoid CSV saving
print("\nAnalyzing seasonal impact on individual stocks...")
seasonal_impact = {}
unique_symbols = combined_df['Ticker'].unique()

# Only process a reasonable number of stocks for demonstration
analyze_symbols = unique_symbols[:min(20, len(unique_symbols))]

for symbol in analyze_symbols:
    symbol_data = combined_df[combined_df['Ticker'] == symbol]
    if len(symbol_data) > 200:  # Only analyze stocks with sufficient data
        print(f"Analyzing {symbol}...")
        symbol_seasonal_impact = {}
        
        for season in range(1, 5):
            season_name = season_map.get(season, f"Season {season}")
            season_data = symbol_data[symbol_data['season'] == season]
            
            if len(season_data) > 50:  # Make sure we have enough data for this season
                # Quick analysis of this specific stock in this season
                try:
                    season_X_base = season_data[base_features]
                    season_X_all = season_data[all_features]
                    season_y = season_data['Target']
                    
                    X_train, X_test, y_train, y_test = train_test_split(season_X_base, season_y, test_size=0.25)
                    base_rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
                    base_rf.fit(X_train, y_train)
                    base_acc = accuracy_score(y_test, base_rf.predict(X_test))
                    
                    X_train, X_test, y_train, y_test = train_test_split(season_X_all, season_y, test_size=0.25)
                    all_rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
                    all_rf.fit(X_train, y_train)
                    all_acc = accuracy_score(y_test, all_rf.predict(X_test))
                    
                    symbol_seasonal_impact[season_name] = all_acc - base_acc
                except Exception as e:
                    print(f"Error analyzing {symbol} in {season_name}: {e}")
        
        seasonal_impact[symbol] = symbol_seasonal_impact

# Convert results to DataFrame for easier analysis
seasonal_results = []
for symbol, impacts in seasonal_impact.items():
    for season, impact in impacts.items():
        seasonal_results.append({
            'Ticker': symbol,
            'Season': season,
            'Weather_Impact': impact
        })

seasonal_impact_df = pd.DataFrame(seasonal_results)

# Find stocks with strongest seasonal weather effects
print("\n--- Stocks Most Affected by Weather by Season ---")
for season in season_map.values():
    season_df = seasonal_impact_df[seasonal_impact_df['Season'] == season]
    if not season_df.empty:
        top_stocks = season_df.sort_values('Weather_Impact', ascending=False).head(5)
        print(f"\n{season}:")
        print(top_stocks)

# Visualize the top stock-season combinations
if len(seasonal_impact_df) > 0:
    plt.figure(figsize=(14, 8))
    top_overall = seasonal_impact_df.sort_values('Weather_Impact', ascending=False).head(15)
    sns.barplot(x='Weather_Impact', y='Ticker', hue='Season', data=top_overall)
    plt.title('Top 15 Stock-Season Combinations with Highest Weather Impact')
    plt.tight_layout()
    plt.savefig(os.path.join(weather_data_dir, 'top_stock_season_impact.png'))
    plt.close()



Analyzing seasonal impact on individual stocks...
Analyzing ABBV...
Analyzing ABT...
Analyzing ADM...
Analyzing AES...
Analyzing AJG...
Analyzing ALB...
Analyzing ALL...
Analyzing ALLE...
Analyzing AMP...
Analyzing AMZN...
Analyzing ANET...
Analyzing ANSS...
Analyzing AOS...
Analyzing APH...
Analyzing AXON...
Analyzing AXP...
Analyzing BAX...
Analyzing BBY...
Analyzing BDX...
Analyzing BEN...

--- Stocks Most Affected by Weather by Season ---

Winter:
   Ticker  Season  Weather_Impact
48    AOS  Winter        0.073913
52    APH  Winter        0.060870
20    ALB  Winter        0.034783
12    AES  Winter        0.030435
68    BBY  Winter        0.000000

Spring:
   Ticker  Season  Weather_Impact
41   ANET  Spring        0.049383
49    AOS  Spring        0.025316
53    APH  Spring        0.021097
77    BEN  Spring        0.012658
17    AJG  Spring        0.000000

Summer:
   Ticker  Season  Weather_Impact
14    AES  Summer        0.054393
42   ANET  Summer        0.045455
22    ALB  Summ

In [16]:
# Cell 16: Save the final dataset with seasonal features - Modified to avoid CSV saving
print("\nSkipping saving of large CSV files...")

print("\nSeasonal analysis complete! Results saved to the weather_data directory.")

# Display summary of findings
print("\n=== SUMMARY OF FINDINGS ===")
print(f"1. Overall improvement from adding weather data: {np.mean(all_cv_scores) - np.mean(base_cv_scores):.4f}")
print("2. Top weather features in importance ranking:")
for feature in weather_in_top[:min(5, len(weather_in_top))]:
    print(f"   - {feature}")
print("3. Seasons with strongest weather impact:")
for season in range(1, 5):
    if seasonal_cv_scores[season]:
        avg_improvement = np.mean(seasonal_cv_scores[season])
        season_name = season_map.get(season, f"Season {season}")
        print(f"   - {season_name}: {avg_improvement:.4f}")

# Calculate overall stock weather sensitivity
if len(seasonal_impact_df) > 0:
    ticker_impact = seasonal_impact_df
    print("4. Top 3 stocks most influenced by weather:")
    top_stocks = seasonal_impact_df.groupby('Ticker')['Weather_Impact'].mean().sort_values(ascending=False).head(3)
    for stock, impact in top_stocks.items():
        print(f"   - {stock}: {impact:.4f}")

# Create a comprehensive report with findings
report = f"""
# Weather Impact on Stock Price Prediction Analysis Report

## Overview
This analysis explored the impact of weather data on stock price prediction accuracy for S&P 500 stocks.
The analysis incorporated seasonal patterns and weather anomalies to determine whether and when
weather conditions influence stock price movements.

## Key Findings

1. **Overall Impact**: Adding weather features {('improved' if np.mean(all_cv_scores) > np.mean(base_cv_scores) else 'did not improve')} prediction accuracy 
   by {abs(np.mean(all_cv_scores) - np.mean(base_cv_scores)):.4f} on average.

2. **Most Important Weather Features**:
   {', '.join(weather_in_top[:min(5, len(weather_in_top))])}

3. **Seasonal Effects**:
"""

for season in range(1, 5):
    if seasonal_cv_scores[season]:
        avg_improvement = np.mean(seasonal_cv_scores[season])
        season_name = season_map.get(season, f"Season {season}")
        report += """
5. **Stock-Season Combinations with Strongest Weather Impact**:
"""
    top_combinations = seasonal_impact_df.sort_values('Weather_Impact', ascending=False).head(5)
    for _, row in top_combinations.iterrows():
        report += f"   - {row['Ticker']} in {row['Season']}: {row['Weather_Impact']:.4f} accuracy improvement\n"

# Save the report
with open(os.path.join(weather_data_dir, 'weather_impact_report.md'), 'w') as f:
    f.write(report)

print("\nFull analysis report has been saved to:", os.path.join(weather_data_dir, 'weather_impact_report.md'))



Skipping saving of large CSV files...

Seasonal analysis complete! Results saved to the weather_data directory.

=== SUMMARY OF FINDINGS ===
1. Overall improvement from adding weather data: 0.0713
2. Top weather features in importance ranking:
   - wind_speed_10m_max
   - wind_speed_10m_max_lag2
   - wind_speed_10m_max_lag1
   - temp_change
   - temp_std_rolling7
3. Seasons with strongest weather impact:
   - Winter: 0.0661
   - Spring: 0.0722
   - Summer: 0.0785
   - Fall: 0.0679
4. Top 3 stocks most influenced by weather:
   - AOS: 0.0196
   - AES: 0.0180
   - ANET: 0.0095

Full analysis report has been saved to: weather_data\weather_impact_report.md


In [17]:
# Cell 17: Enhanced trading simulation to estimate actual gains - Fixed
print("\n--- Running Enhanced Trading Simulation ---")

def enhanced_trading_simulation(model, X, ticker_data, feature_set_name, initial_investment=10000):
    """
    Run a more realistic trading simulation with portfolio tracking
    
    Parameters:
    -----------
    model : sklearn model
        Trained prediction model
    X : DataFrame
        Feature data
    ticker_data : DataFrame
        Stock data with dates and prices
    feature_set_name : str
        Name of the feature set used
    initial_investment : float
        Initial investment amount
        
    Returns:
    --------
    portfolio_df : DataFrame
        DataFrame with portfolio performance
    metrics : dict
        Performance metrics
    """
    from sklearn.preprocessing import StandardScaler
    import matplotlib.dates as mdates
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import os
    
    print(f"\nRunning enhanced simulation for {feature_set_name}...")
    
    # Scale features
    try:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Get predictions
        y_pred = model.predict(X_scaled)
        y_pred_proba = model.predict_proba(X_scaled)[:, 1]  # Probability of class 1
    except Exception as e:
        print(f"Error during prediction: {e}")
        return None, None
    
    # Create signals dataframe
    signals_df = pd.DataFrame({
        'Date': ticker_data['Date'],
        'Ticker': ticker_data['Ticker'],
        'Close': ticker_data['Close'],
        'Predicted': y_pred,
        'Confidence': y_pred_proba
    })
    
    # Convert signals to Buy/Sell/Hold
    # Use probability threshold to determine confidence level
    signals_df['Signal'] = 'Hold'
    signals_df.loc[signals_df['Confidence'] > 0.7, 'Signal'] = 'Buy'
    signals_df.loc[signals_df['Confidence'] < 0.3, 'Signal'] = 'Sell'
    
    # Sort by date for chronological simulation
    signals_df = signals_df.sort_values(['Date', 'Ticker'])
    
    # Group by ticker to simulate trading for each stock separately
    portfolio_results = []
    
    for ticker in signals_df['Ticker'].unique():
        ticker_signals = signals_df[signals_df['Ticker'] == ticker].copy()
        if len(ticker_signals) < 30:  # Skip if not enough data points
            continue
            
        # Initialize portfolio tracking
        ticker_signals['Position'] = 0  # 0 = no position, 1 = long position
        ticker_signals['Portfolio_Value'] = initial_investment
        ticker_signals['Cash'] = initial_investment
        ticker_signals['Shares'] = 0
        ticker_signals['Trade'] = 'None'
        ticker_signals['Trade_Price'] = np.nan
        ticker_signals['Return'] = 0.0
        
        # Simulation with slippage and transaction costs
        transaction_cost_pct = 0.001  # 0.1% per trade
        slippage_pct = 0.001  # 0.1% slippage
        
        for i in range(1, len(ticker_signals)):
            prev = ticker_signals.iloc[i-1]
            curr = ticker_signals.iloc[i]
            
            # Default: carry forward previous position and values
            ticker_signals.loc[ticker_signals.index[i], 'Position'] = prev['Position']
            ticker_signals.loc[ticker_signals.index[i], 'Cash'] = prev['Cash']
            ticker_signals.loc[ticker_signals.index[i], 'Shares'] = prev['Shares']
            
            # Trading logic with signals
            if prev['Signal'] == 'Buy' and prev['Position'] == 0:
                # Buy with slippage
                buy_price = prev['Close'] * (1 + slippage_pct)
                transaction_cost = prev['Cash'] * transaction_cost_pct
                available_cash = prev['Cash'] - transaction_cost
                shares_to_buy = available_cash / buy_price
                
                # Update position
                ticker_signals.loc[ticker_signals.index[i], 'Position'] = 1
                ticker_signals.loc[ticker_signals.index[i], 'Cash'] = 0
                ticker_signals.loc[ticker_signals.index[i], 'Shares'] = shares_to_buy
                ticker_signals.loc[ticker_signals.index[i], 'Trade'] = 'Buy'
                ticker_signals.loc[ticker_signals.index[i], 'Trade_Price'] = buy_price
                
            elif prev['Signal'] == 'Sell' and prev['Position'] == 1:
                # Sell with slippage
                sell_price = prev['Close'] * (1 - slippage_pct)
                shares_value = prev['Shares'] * sell_price
                transaction_cost = shares_value * transaction_cost_pct
                cash_after_sale = shares_value - transaction_cost
                
                # Update position
                ticker_signals.loc[ticker_signals.index[i], 'Position'] = 0
                ticker_signals.loc[ticker_signals.index[i], 'Cash'] = cash_after_sale
                ticker_signals.loc[ticker_signals.index[i], 'Shares'] = 0
                ticker_signals.loc[ticker_signals.index[i], 'Trade'] = 'Sell'
                ticker_signals.loc[ticker_signals.index[i], 'Trade_Price'] = sell_price
            
            # Calculate current portfolio value
            if ticker_signals.loc[ticker_signals.index[i], 'Position'] == 1:
                # Long position: value is in shares
                shares_value = ticker_signals.loc[ticker_signals.index[i], 'Shares'] * curr['Close']
                ticker_signals.loc[ticker_signals.index[i], 'Portfolio_Value'] = shares_value
            else:
                # No position: value is cash
                ticker_signals.loc[ticker_signals.index[i], 'Portfolio_Value'] = ticker_signals.loc[ticker_signals.index[i], 'Cash']
            
            # Calculate daily return
            ticker_signals.loc[ticker_signals.index[i], 'Return'] = (
                ticker_signals.loc[ticker_signals.index[i], 'Portfolio_Value'] / 
                ticker_signals.loc[ticker_signals.index[i-1], 'Portfolio_Value'] - 1
            )
        
        # Add to combined results
        portfolio_results.append(ticker_signals)
    
    # Combine all portfolio results
    if portfolio_results:
        try:
            portfolio_df = pd.concat(portfolio_results, ignore_index=True)
            
            # Calculate performance metrics
            total_return = (portfolio_df.groupby('Ticker')['Portfolio_Value'].last() / initial_investment - 1).mean()
            
            # Calculate Sharpe ratio with error handling
            try:
                sharpe_ratio = portfolio_df.groupby('Ticker')['Return'].mean() / portfolio_df.groupby('Ticker')['Return'].std() * np.sqrt(252)  # Annualized
                mean_sharpe = sharpe_ratio.mean()
            except Exception as e:
                print(f"Error calculating Sharpe ratio: {e}")
                mean_sharpe = float('nan')
            
            # Calculate drawdowns
            try:
                for ticker in portfolio_df['Ticker'].unique():
                    ticker_mask = portfolio_df['Ticker'] == ticker
                    portfolio_df.loc[ticker_mask, 'Cum_Return'] = (1 + portfolio_df.loc[ticker_mask, 'Return']).cumprod()
                    portfolio_df.loc[ticker_mask, 'Peak'] = portfolio_df.loc[ticker_mask, 'Cum_Return'].cummax()
                    portfolio_df.loc[ticker_mask, 'Drawdown'] = (portfolio_df.loc[ticker_mask, 'Cum_Return'] / portfolio_df.loc[ticker_mask, 'Peak']) - 1
                
                max_drawdown = portfolio_df.groupby('Ticker')['Drawdown'].min().mean()
            except Exception as e:
                print(f"Error calculating drawdowns: {e}")
                max_drawdown = float('nan')
            
            # Calculate win rate - THIS IS THE FIXED PART
            try:
                # Filter sell trades
                sell_trades = portfolio_df[portfolio_df['Trade'] == 'Sell']
                # Calculate previous cash values (shifted)
                previous_cash = portfolio_df['Cash'].shift(1)
                # Filter profitable sells (portfolio value > previous cash)
                profitable_sells = portfolio_df[(portfolio_df['Trade'] == 'Sell') & 
                                               (portfolio_df['Portfolio_Value'] > previous_cash)]
                
                # Calculate win rate
                win_rate = len(profitable_sells) / max(1, len(sell_trades))
            except Exception as e:
                print(f"Error calculating win rate: {e}")
                win_rate = float('nan')
            
            # Count trades
            n_trades = len(portfolio_df[portfolio_df['Trade'] != 'None'])
            
            metrics = {
                'total_return': total_return,
                'annualized_return': total_return * (252 / max(1, len(portfolio_df['Date'].unique()))),
                'sharpe_ratio': mean_sharpe,
                'max_drawdown': max_drawdown,
                'win_rate': win_rate,
                'n_trades': n_trades
            }
            
            # Print performance metrics
            print(f"\n--- {feature_set_name} Trading Simulation Results ---")
            print(f"Total Return: {metrics['total_return']:.4f} ({metrics['total_return'] * 100:.2f}%)")
            print(f"Annualized Return: {metrics['annualized_return']:.4f} ({metrics['annualized_return'] * 100:.2f}%)")
            
            if not np.isnan(metrics['sharpe_ratio']):
                print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.4f}")
            else:
                print("Sharpe Ratio: Not available")
                
            print(f"Max Drawdown: {metrics['max_drawdown']:.4f} ({metrics['max_drawdown'] * 100:.2f}%)")
            print(f"Win Rate: {metrics['win_rate']:.4f} ({metrics['win_rate'] * 100:.2f}%)")
            print(f"Number of Trades: {metrics['n_trades']}")
            
            # Plot portfolio performance for top performers
            try:
                top_performers = portfolio_df.groupby('Ticker')['Portfolio_Value'].last() / initial_investment
                top_performers = top_performers.sort_values(ascending=False).head(5)
                
                for ticker in top_performers.index:
                    ticker_data = portfolio_df[portfolio_df['Ticker'] == ticker].copy()
                    
                    plt.figure(figsize=(12, 8))
                    
                    # Portfolio value over time
                    plt.subplot(2, 1, 1)
                    plt.plot(ticker_data['Date'], ticker_data['Portfolio_Value'], label='Portfolio Value')
                    
                    # Mark buy/sell points
                    buys = ticker_data[ticker_data['Trade'] == 'Buy']
                    sells = ticker_data[ticker_data['Trade'] == 'Sell']
                    
                    plt.scatter(buys['Date'], buys['Portfolio_Value'], marker='^', color='green', s=100, label='Buy')
                    plt.scatter(sells['Date'], sells['Portfolio_Value'], marker='v', color='red', s=100, label='Sell')
                    
                    plt.title(f'{ticker} Portfolio Performance - {feature_set_name}')
                    plt.ylabel('Portfolio Value ($)')
                    plt.legend()
                    plt.grid(True, alpha=0.3)
                    
                    # Drawdown chart
                    plt.subplot(2, 1, 2)
                    plt.fill_between(ticker_data['Date'], ticker_data['Drawdown'] * 100, 0, color='red', alpha=0.3)
                    plt.plot(ticker_data['Date'], ticker_data['Drawdown'] * 100, color='red', label='Drawdown %')
                    
                    plt.title(f'{ticker} Drawdown')
                    plt.ylabel('Drawdown (%)')
                    plt.xlabel('Date')
                    plt.legend()
                    plt.grid(True, alpha=0.3)
                    
                    plt.tight_layout()
                    plt.savefig(os.path.join(weather_data_dir, f'{ticker}_{feature_set_name.replace(" ", "_").lower()}_performance.png'))
                    plt.close()
                
                # Compare portfolio growth across all tickers
                plt.figure(figsize=(14, 8))
                for ticker in portfolio_df['Ticker'].unique():
                    ticker_data = portfolio_df[portfolio_df['Ticker'] == ticker]
                    if len(ticker_data) > 0:
                        plt.plot(ticker_data['Date'], ticker_data['Portfolio_Value'], label=ticker, alpha=0.7)
                
                plt.axhline(y=initial_investment, color='k', linestyle='-', alpha=0.3)
                plt.title(f'Portfolio Growth Comparison - {feature_set_name}')
                plt.ylabel('Portfolio Value ($)')
                plt.xlabel('Date')
                plt.grid(True, alpha=0.3)
                plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
                plt.tight_layout()
                plt.savefig(os.path.join(weather_data_dir, f'portfolio_comparison_{feature_set_name.replace(" ", "_").lower()}.png'))
                plt.close()
            except Exception as e:
                print(f"Error in plotting: {e}")
            
            return portfolio_df, metrics
        except Exception as e:
            print(f"Error in performance calculation: {e}")
            return None, None
    else:
        print("No valid trading data for simulation.")
        return None, None

# Run simulations with both models for comparison
try:
    # Select specific tickers for more detailed simulation to avoid excessive computation
    top_tickers = combined_df['Ticker'].value_counts().head(5).index.tolist()
    print(f"Selected top tickers for simulation: {top_tickers}")
    simulation_data = combined_df[combined_df['Ticker'].isin(top_tickers)].copy()
    
    if len(simulation_data) < 100:
        print(f"Warning: Only {len(simulation_data)} data points for selected tickers. Selecting more tickers...")
        top_tickers = combined_df['Ticker'].value_counts().head(10).index.tolist()
        simulation_data = combined_df[combined_df['Ticker'].isin(top_tickers)].copy()
    
    print(f"Simulation data shape: {simulation_data.shape}")
    
    # Base model simulation (stock data only)
    print("\nRunning base model trading simulation...")
    base_X = simulation_data[base_features]
    base_portfolio, base_metrics = enhanced_trading_simulation(
        base_model, base_X, simulation_data, "Base Model", initial_investment=10000
    )
    
    # Combined model simulation (stock + weather)
    print("\nRunning combined model trading simulation...")
    all_X = simulation_data[all_features]
    all_portfolio, all_metrics = enhanced_trading_simulation(
        all_model, all_X, simulation_data, "Combined Model", initial_investment=10000
    )
    
    # Compare trading performance between models
    if base_metrics and all_metrics:
        try:
            # Create comparison dataframe for key metrics
            metrics_comparison = pd.DataFrame({
                'Metric': list(base_metrics.keys()),
                'Base Model': list(base_metrics.values()),
                'Combined Model': list(all_metrics.values()),
                'Improvement': [all_metrics[key] - base_metrics[key] for key in base_metrics.keys()]
            })
            
            # Convert to string representations for display
            metrics_comparison_display = metrics_comparison.copy()
            
            # Format as percentages where appropriate
            percent_metrics = ['total_return', 'annualized_return', 'max_drawdown', 'win_rate']
            for metric in percent_metrics:
                try:
                    idx = metrics_comparison_display.index[metrics_comparison_display['Metric'] == metric][0]
                    metrics_comparison_display.loc[idx, 'Base Model'] = f"{base_metrics[metric] * 100:.2f}%"
                    metrics_comparison_display.loc[idx, 'Combined Model'] = f"{all_metrics[metric] * 100:.2f}%"
                    metrics_comparison_display.loc[idx, 'Improvement'] = f"{(all_metrics[metric] - base_metrics[metric]) * 100:.2f}%"
                except Exception as e:
                    print(f"Error formatting {metric}: {e}")
            
            print("\n--- Trading Performance Comparison ---")
            print(metrics_comparison_display)
            
            # Visualize performance comparison
            try:
                plt.figure(figsize=(12, 8))
                
                # Create bar chart for key numerical metrics
                numerical_metrics = ['sharpe_ratio', 'n_trades']
                metrics_df = pd.DataFrame({
                    'Metric': numerical_metrics,
                    'Base Model': [base_metrics[m] for m in numerical_metrics],
                    'Combined Model': [all_metrics[m] for m in numerical_metrics]
                })
                
                # Reshape for plotting
                plot_df = pd.melt(metrics_df, id_vars=['Metric'], var_name='Model', value_name='Value')
                
                # Plot
                import seaborn as sns
                sns.barplot(x='Metric', y='Value', hue='Model', data=plot_df)
                plt.title('Trading Performance Metrics Comparison')
                plt.grid(True, alpha=0.3)
                plt.savefig(os.path.join(weather_data_dir, 'trading_metrics_comparison.png'))
                plt.close()
            except Exception as e:
                print(f"Error creating metrics comparison plot: {e}")
            
            # Plot cumulative returns comparison - average across all tickers for each model
            if base_portfolio is not None and all_portfolio is not None:
                try:
                    # Calculate mean portfolio value by date for each model
                    base_daily = base_portfolio.groupby(['Date'])['Portfolio_Value'].mean().reset_index()
                    base_daily['Relative_Value'] = base_daily['Portfolio_Value'] / 10000
                    
                    all_daily = all_portfolio.groupby(['Date'])['Portfolio_Value'].mean().reset_index()
                    all_daily['Relative_Value'] = all_daily['Portfolio_Value'] / 10000
                    
                    # Plot comparison
                    plt.figure(figsize=(12, 8))
                    plt.plot(base_daily['Date'], base_daily['Relative_Value'], label='Base Model', color='blue')
                    plt.plot(all_daily['Date'], all_daily['Relative_Value'], label='Combined Model', color='green')
                    
                    plt.axhline(y=1, color='k', linestyle='-', alpha=0.3)
                    plt.title('Average Cumulative Return Comparison')
                    plt.ylabel('Portfolio Value (Relative to Initial Investment)')
                    plt.xlabel('Date')
                    plt.legend()
                    plt.grid(True, alpha=0.3)
                    plt.tight_layout()
                    plt.savefig(os.path.join(weather_data_dir, 'cumulative_return_comparison.png'))
                    plt.close()
                except Exception as e:
                    print(f"Error creating cumulative returns plot: {e}")
        except Exception as e:
            print(f"Error in metrics comparison: {e}")
    else:
        print("Could not compare metrics: one or both simulations failed.")
except Exception as e:
    print(f"Error in simulation setup: {e}")


--- Running Enhanced Trading Simulation ---
Selected top tickers for simulation: ['IFF', 'K', 'KMX', 'LEN', 'LH']
Simulation data shape: (18840, 100)

Running base model trading simulation...

Running enhanced simulation for Base Model...

--- Base Model Trading Simulation Results ---
Total Return: 4.1598 (415.98%)
Annualized Return: 0.2782 (27.82%)
Sharpe Ratio: 0.4653
Max Drawdown: -0.4372 (-43.72%)
Win Rate: 1.0000 (100.00%)
Number of Trades: 63

Running combined model trading simulation...

Running enhanced simulation for Combined Model...

--- Combined Model Trading Simulation Results ---
Total Return: 234.8231 (23482.31%)
Annualized Return: 15.7047 (1570.47%)
Sharpe Ratio: 1.3630
Max Drawdown: -0.3549 (-35.49%)
Win Rate: 1.0000 (100.00%)
Number of Trades: 1046

--- Trading Performance Comparison ---
              Metric Base Model Combined Model Improvement
0       total_return    415.98%      23482.31%   23066.33%
1  annualized_return     27.82%       1570.47%    1542.65%
2    

In [18]:
# Cell 19: Add Simple Moving Average (SMA) Trading Strategy as Benchmark
print("\n--- Simple Moving Average (SMA) Trading Strategy Benchmark ---")

def sma_trading_strategy(combined_df, weather_sensitive_stocks):
    """
    Implement a Simple Moving Average (SMA) trading strategy as a benchmark
    
    Parameters:
    -----------
    combined_df : DataFrame
        Combined stock and weather data
    weather_sensitive_stocks : list
        List of tickers to analyze (same as weather-sensitive strategy)
        
    Returns:
    --------
    strategy_df : DataFrame
        DataFrame with strategy performance
    """
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import os
    
    print("\nImplementing SMA trading strategy benchmark...")
    
    # Parameters for SMA strategy
    short_window = 20  # Short moving average window (days)
    long_window = 50   # Long moving average window (days)
    
    print(f"Using SMA parameters: short_window={short_window}, long_window={long_window}")
    
    # Filter data to include only the stocks we're analyzing
    strategy_data = combined_df[combined_df['Ticker'].isin(weather_sensitive_stocks)].copy()
    
    # Strategy results and performance summary
    strategy_results = []
    performance_summary = []
    
    for ticker in weather_sensitive_stocks:
        print(f"\nRunning SMA strategy for {ticker}...")
        ticker_data = strategy_data[strategy_data['Ticker'] == ticker].copy()
        
        if len(ticker_data) < long_window + 10:
            print(f"Not enough data for {ticker}, skipping...")
            continue
        
        # Sort by date for calculations
        ticker_data = ticker_data.sort_values('Date')
        
        # Calculate moving averages
        ticker_data['SMA_Short'] = ticker_data['Close'].rolling(window=short_window, min_periods=1).mean()
        ticker_data['SMA_Long'] = ticker_data['Close'].rolling(window=long_window, min_periods=1).mean()
        
        # Generate signals: 1 = Buy, 0 = Sell
        ticker_data['Signal'] = 0
        ticker_data.loc[ticker_data['SMA_Short'] > ticker_data['SMA_Long'], 'Signal'] = 1
        
        # Create a new column for signal changes
        ticker_data['Position'] = ticker_data['Signal'].diff()
        
        # Initialize trading simulation
        initial_investment = 10000
        ticker_data['SMA_Cash'] = initial_investment
        ticker_data['SMA_Shares'] = 0
        ticker_data['SMA_Portfolio'] = initial_investment
        ticker_data['SMA_Return'] = 0.0
        ticker_data['SMA_Trade'] = 'None'
        
        # Parameters for trading costs
        transaction_cost_pct = 0.001  # 0.1% per trade
        slippage_pct = 0.001  # 0.1% slippage
        
        # Backtest strategy
        position = 0
        for i in range(1, len(ticker_data)):
            prev_row = ticker_data.iloc[i-1]
            curr_row = ticker_data.iloc[i]
            
            # Default: carry forward previous values
            ticker_data.loc[ticker_data.index[i], 'SMA_Cash'] = prev_row['SMA_Cash']
            ticker_data.loc[ticker_data.index[i], 'SMA_Shares'] = prev_row['SMA_Shares']
            
            # Check for signal changes
            if prev_row['Position'] == 1:  # Buy signal
                # Buy with slippage
                buy_price = prev_row['Close'] * (1 + slippage_pct)
                transaction_cost = prev_row['SMA_Cash'] * transaction_cost_pct
                shares_to_buy = (prev_row['SMA_Cash'] - transaction_cost) / buy_price if buy_price > 0 else 0
                
                # Update position
                ticker_data.loc[ticker_data.index[i], 'SMA_Cash'] = 0
                ticker_data.loc[ticker_data.index[i], 'SMA_Shares'] = shares_to_buy
                ticker_data.loc[ticker_data.index[i], 'SMA_Trade'] = 'Buy'
                
                position = 1
            
            elif prev_row['Position'] == -1:  # Sell signal
                # Sell with slippage
                sell_price = prev_row['Close'] * (1 - slippage_pct)
                shares_value = prev_row['SMA_Shares'] * sell_price
                transaction_cost = shares_value * transaction_cost_pct
                
                # Update position
                ticker_data.loc[ticker_data.index[i], 'SMA_Cash'] = shares_value - transaction_cost
                ticker_data.loc[ticker_data.index[i], 'SMA_Shares'] = 0
                ticker_data.loc[ticker_data.index[i], 'SMA_Trade'] = 'Sell'
                
                position = 0
            
            # Calculate current portfolio value
            if position == 1:
                shares_value = ticker_data.loc[ticker_data.index[i], 'SMA_Shares'] * curr_row['Close']
                ticker_data.loc[ticker_data.index[i], 'SMA_Portfolio'] = shares_value
            else:
                ticker_data.loc[ticker_data.index[i], 'SMA_Portfolio'] = ticker_data.loc[ticker_data.index[i], 'SMA_Cash']
            
            # Calculate daily return
            if prev_row['SMA_Portfolio'] > 0:
                ticker_data.loc[ticker_data.index[i], 'SMA_Return'] = (
                    ticker_data.loc[ticker_data.index[i], 'SMA_Portfolio'] / 
                    prev_row['SMA_Portfolio'] - 1
                )
            else:
                ticker_data.loc[ticker_data.index[i], 'SMA_Return'] = 0
        
        # Calculate cumulative returns
        ticker_data['SMA_Cum_Return'] = (1 + ticker_data['SMA_Return']).cumprod()
        
        # Calculate buy-and-hold returns for comparison
        ticker_data['BuyHold_Return'] = ticker_data['Close'] / ticker_data['Close'].iloc[0]
        
        # Calculate final returns
        final_sma_return = ticker_data['SMA_Portfolio'].iloc[-1] / initial_investment - 1
        final_buy_hold = ticker_data['BuyHold_Return'].iloc[-1] - 1
        
        # Calculate risk metrics
        sma_vol = ticker_data['SMA_Return'].std() * np.sqrt(252)  # Annualized
        sma_sharpe = (final_sma_return / len(ticker_data) * 252) / max(sma_vol, 0.0001)  # Avoid division by zero
        
        # Count trades
        sma_trades = len(ticker_data[ticker_data['SMA_Trade'] != 'None'])
        
        # Add to results
        strategy_results.append(ticker_data)
        
        # Display performance summary
        print(f"\n{ticker} SMA Strategy Performance Summary:")
        print(f"SMA Strategy Return: {final_sma_return:.4f} ({final_sma_return * 100:.2f}%)")
        print(f"Buy and Hold Return: {final_buy_hold:.4f} ({final_buy_hold * 100:.2f}%)")
        print(f"SMA Sharpe Ratio: {sma_sharpe:.4f}")
        print(f"Total Trades: {sma_trades}")
        
        # Create visualization
        try:
            plt.figure(figsize=(14, 10))
            
            # Price and moving averages
            plt.subplot(3, 1, 1)
            plt.plot(ticker_data['Date'], ticker_data['Close'], label='Close Price', alpha=0.6)
            plt.plot(ticker_data['Date'], ticker_data['SMA_Short'], label=f'SMA ({short_window})', color='orange')
            plt.plot(ticker_data['Date'], ticker_data['SMA_Long'], label=f'SMA ({long_window})', color='blue')
            
            # Mark buy/sell points
            buys = ticker_data[ticker_data['SMA_Trade'] == 'Buy']
            sells = ticker_data[ticker_data['SMA_Trade'] == 'Sell']
            
            if len(buys) > 0:
                plt.scatter(buys['Date'], buys['Close'], marker='^', color='green', s=100, label='Buy Signal')
            if len(sells) > 0:
                plt.scatter(sells['Date'], sells['Close'], marker='v', color='red', s=100, label='Sell Signal')
            
            plt.title(f'{ticker} Price and Moving Averages')
            plt.ylabel('Price ($)')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            # Portfolio value
            plt.subplot(3, 1, 2)
            plt.plot(ticker_data['Date'], ticker_data['SMA_Portfolio'], label='SMA Strategy', color='blue')
            plt.axhline(y=initial_investment, color='k', linestyle='--', alpha=0.3, label='Initial Investment')
            
            plt.title(f'{ticker} SMA Strategy Portfolio Value')
            plt.ylabel('Portfolio Value ($)')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            # Cumulative returns comparison
            plt.subplot(3, 1, 3)
            plt.plot(ticker_data['Date'], ticker_data['SMA_Cum_Return'], label='SMA Strategy', color='blue')
            plt.plot(ticker_data['Date'], ticker_data['BuyHold_Return'], label='Buy and Hold', color='green', linestyle=':')
            
            plt.title(f'{ticker} Cumulative Returns Comparison')
            plt.ylabel('Cumulative Return (Starting = 1)')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.savefig(os.path.join(weather_data_dir, f'{ticker}_sma_strategy.png'))
            plt.close()
        except Exception as e:
            print(f"Error creating visualization: {e}")
        
        # Add to performance summary
        performance_summary.append({
            'Ticker': ticker,
            'SMA_Return': final_sma_return,
            'BuyHold_Return': final_buy_hold,
            'Outperformance': final_sma_return - final_buy_hold,
            'SMA_Sharpe': sma_sharpe,
            'SMA_Trades': sma_trades
        })
    
    # Combine all results
    if strategy_results and len(strategy_results) > 0:
        try:
            all_results = pd.concat(strategy_results, ignore_index=True)
            
            # Create summary dataframe
            if performance_summary:
                summary_df = pd.DataFrame(performance_summary)
                
                # Print overall results
                print("\n=== Overall SMA Strategy Results ===")
                print(f"Average SMA Strategy Return: {summary_df['SMA_Return'].mean():.4f} ({summary_df['SMA_Return'].mean() * 100:.2f}%)")
                print(f"Average Buy & Hold Return: {summary_df['BuyHold_Return'].mean():.4f} ({summary_df['BuyHold_Return'].mean() * 100:.2f}%)")
                print(f"Average Outperformance: {summary_df['Outperformance'].mean():.4f} ({summary_df['Outperformance'].mean() * 100:.2f}%)")
                print(f"Average SMA Sharpe Ratio: {summary_df['SMA_Sharpe'].mean():.4f}")
                print(f"Average Number of Trades: {summary_df['SMA_Trades'].mean():.1f}")
                
                # Plot summary across tickers
                try:
                    plt.figure(figsize=(12, 8))
                    
                    x = np.arange(len(summary_df))
                    width = 0.35
                    
                    plt.bar(x - width/2, summary_df['SMA_Return'] * 100, width, label='SMA Strategy')
                    plt.bar(x + width/2, summary_df['BuyHold_Return'] * 100, width, label='Buy & Hold')
                    
                    plt.xlabel('Ticker')
                    plt.ylabel('Return (%)')
                    plt.title('SMA Strategy vs Buy & Hold Returns by Ticker')
                    plt.xticks(x, summary_df['Ticker'])
                    plt.legend()
                    plt.grid(True, alpha=0.3)
                    
                    plt.tight_layout()
                    plt.savefig(os.path.join(weather_data_dir, 'sma_strategy_comparison.png'))
                    plt.close()
                except Exception as e:
                    print(f"Error creating summary plots: {e}")
                
                return all_results, summary_df
            else:
                return all_results, None
        except Exception as e:
            print(f"Error combining results: {e}")
            return None, None
    else:
        print("No strategy results collected.")
        return None, None

# Run the SMA trading strategy
try:
    # Use the same stocks as in the weather-sensitive strategy
    if 'seasonal_impact_df' in globals() and len(seasonal_impact_df) > 0:
        weather_sensitive_stocks = seasonal_impact_df.groupby('Ticker')['Weather_Impact'].mean()
        weather_sensitive_stocks = weather_sensitive_stocks.sort_values(ascending=False).head(5).index.tolist()
    else:
        # Fallback: use top 5 stocks by data volume
        weather_sensitive_stocks = combined_df['Ticker'].value_counts().head(5).index.tolist()
    
    print(f"Analyzing SMA strategy for tickers: {weather_sensitive_stocks}")
    
    # Run SMA strategy
    sma_results, sma_summary = sma_trading_strategy(combined_df, weather_sensitive_stocks)
except Exception as e:
    print(f"Error running SMA strategy: {e}")
    sma_results, sma_summary = None, None

# FIXED COMPARISON CODE: Check if both summaries exist before comparing
# Define strategy_summary if it doesn't exist (from fixed_summary if available)
if 'strategy_summary' not in globals() and 'fixed_summary' in globals():
    strategy_summary = fixed_summary
    print("Using fixed_summary as strategy_summary for comparison")
elif 'strategy_summary' not in globals():
    print("Warning: strategy_summary is not defined. Skipping strategy comparison.")
    strategy_summary = None

# Compare SMA strategy with weather-sensitive and base strategies
if (sma_summary is not None and strategy_summary is not None and 
    len(sma_summary) > 0 and len(strategy_summary) > 0):
    
    try:
        print("\n--- Comparing All Strategies ---")
        
        
        # Merge summaries
        comparison_data = []
        
        for ticker in weather_sensitive_stocks:
            sma_data = sma_summary[sma_summary['Ticker'] == ticker]
            weather_data = strategy_summary[strategy_summary['Ticker'] == ticker]
            
            if len(sma_data) > 0 and len(weather_data) > 0:
                comparison_data.append({
                    'Ticker': ticker,
                    'Weather_Return': weather_data['Weather_Return'].values[0],
                    'Basic_Return': weather_data['Basic_Return'].values[0],
                    'SMA_Return': sma_data['SMA_Return'].values[0],
                    'BuyHold_Return': sma_data['BuyHold_Return'].values[0],
                    'Weather_Sharpe': weather_data['Weather_Sharpe'].values[0],
                    'Basic_Sharpe': weather_data['Basic_Sharpe'].values[0],
                    'SMA_Sharpe': sma_data['SMA_Sharpe'].values[0]
                })
        
        if comparison_data:
            # Create comparison dataframe
            comparison_df = pd.DataFrame(comparison_data)
            
            # Print comparison summary
            print("\n=== Strategy Comparison by Average Returns ===")
            print(f"Weather-Sensitive Strategy: {comparison_df['Weather_Return'].mean():.4f} ({comparison_df['Weather_Return'].mean() * 100:.2f}%)")
            print(f"Basic Strategy: {comparison_df['Basic_Return'].mean():.4f} ({comparison_df['Basic_Return'].mean() * 100:.2f}%)")
            print(f"SMA Strategy: {comparison_df['SMA_Return'].mean():.4f} ({comparison_df['SMA_Return'].mean() * 100:.2f}%)")
            print(f"Buy and Hold: {comparison_df['BuyHold_Return'].mean():.4f} ({comparison_df['BuyHold_Return'].mean() * 100:.2f}%)")
            
            print("\n=== Strategy Comparison by Average Sharpe Ratio ===")
            print(f"Weather-Sensitive Strategy: {comparison_df['Weather_Sharpe'].mean():.4f}")
            print(f"Basic Strategy: {comparison_df['Basic_Sharpe'].mean():.4f}")
            print(f"SMA Strategy: {comparison_df['SMA_Sharpe'].mean():.4f}")
            
            # Create comprehensive comparison chart
            try:
                plt.figure(figsize=(14, 10))
                
                # Return comparison
                plt.subplot(2, 1, 1)
                x = np.arange(len(comparison_df))
                width = 0.2
                
                plt.bar(x - width*1.5, comparison_df['Weather_Return'] * 100, width, label='Weather Strategy')
                plt.bar(x - width/2, comparison_df['Basic_Return'] * 100, width, label='Basic Strategy')
                plt.bar(x + width/2, comparison_df['SMA_Return'] * 100, width, label='SMA Strategy')
                plt.bar(x + width*1.5, comparison_df['BuyHold_Return'] * 100, width, label='Buy & Hold')
                
                plt.xlabel('Ticker')
                plt.ylabel('Return (%)')
                plt.title('All Strategies Return Comparison by Ticker')
                plt.xticks(x, comparison_df['Ticker'])
                plt.legend()
                plt.grid(True, alpha=0.3)
                
                # Sharpe ratio comparison
                plt.subplot(2, 1, 2)
                
                plt.bar(x - width, comparison_df['Weather_Sharpe'], width, label='Weather Strategy')
                plt.bar(x, comparison_df['Basic_Sharpe'], width, label='Basic Strategy')
                plt.bar(x + width, comparison_df['SMA_Sharpe'], width, label='SMA Strategy')
                
                plt.xlabel('Ticker')
                plt.ylabel('Sharpe Ratio')
                plt.title('Sharpe Ratio Comparison by Ticker')
                plt.xticks(x, comparison_df['Ticker'])
                plt.legend()
                plt.grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.savefig(os.path.join(weather_data_dir, 'all_strategies_comparison.png'))
                plt.close()
                
                # Create a summary table for the impact report
                strategy_table = pd.DataFrame({
                    'Strategy': ['Weather-Sensitive', 'Basic (No Weather)', 'SMA', 'Buy and Hold'],
                    'Avg Return (%)': [
                        f"{comparison_df['Weather_Return'].mean() * 100:.2f}%",
                        f"{comparison_df['Basic_Return'].mean() * 100:.2f}%",
                        f"{comparison_df['SMA_Return'].mean() * 100:.2f}%",
                        f"{comparison_df['BuyHold_Return'].mean() * 100:.2f}%"
                    ],
                    'Avg Sharpe': [
                        f"{comparison_df['Weather_Sharpe'].mean():.2f}",
                        f"{comparison_df['Basic_Sharpe'].mean():.2f}",
                        f"{comparison_df['SMA_Sharpe'].mean():.2f}",
                        "N/A"
                    ]
                })
                
                # Add strategy comparison to the impact report
                with open(os.path.join(weather_data_dir, 'weather_impact_report.md'), 'a') as f:
                    f.write("\n\n## Strategy Comparison\n\n")
                    f.write("We compared our weather-sensitive trading strategy against several baselines:\n\n")
                    f.write("1. **Basic Model**: Uses the same prediction model but without weather features\n")
                    f.write("2. **Simple Moving Average (SMA)**: A technical analysis strategy using short and long-term moving averages\n")
                    f.write("3. **Buy and Hold**: Simple strategy of buying at the beginning of the period and holding until the end\n\n")
                    f.write("### Performance Summary\n\n")
                    f.write(strategy_table.to_markdown(index=False))
                    f.write("\n\n")
                    
                    # Add interpretation based on results
                    weather_return = comparison_df['Weather_Return'].mean() * 100
                    basic_return = comparison_df['Basic_Return'].mean() * 100
                    sma_return = comparison_df['SMA_Return'].mean() * 100
                    buyhold_return = comparison_df['BuyHold_Return'].mean() * 100
                    
                    f.write("### Interpretation\n\n")
                    
                    if weather_return > max(basic_return, sma_return, buyhold_return):
                        f.write("The weather-sensitive strategy outperformed all baseline strategies, demonstrating that incorporating weather data provides a significant advantage in stock trading. This confirms our hypothesis that weather conditions influence market behavior in ways that can be exploited for trading advantage.\n\n")
                    elif weather_return > basic_return:
                        f.write("The weather-sensitive strategy outperformed the basic model but not all baseline strategies. This suggests that weather data does provide some valuable signal, but may need to be combined with other technical indicators for optimal performance.\n\n")
                    else:
                        f.write("The weather-sensitive strategy did not outperform the baseline strategies in our test period. This suggests that either the relationship between weather and stock performance is weaker than hypothesized, or our current implementation does not optimally capture this relationship.\n\n")
                    
                    f.write("These results suggest that " + (
                        "weather data should be considered a valuable feature in trading systems." if weather_return > basic_return else 
                        "further refinement is needed to effectively leverage weather data in trading systems."
                    ))
                
                print("Strategy comparison completed and added to impact report.")
            except Exception as e:
                print(f"Error creating comparison visualization: {e}")
    except Exception as e:
        print(f"Error comparing strategies: {e}")


--- Simple Moving Average (SMA) Trading Strategy Benchmark ---
Analyzing SMA strategy for tickers: ['AOS', 'AES', 'ANET', 'ALL', 'APH']

Implementing SMA trading strategy benchmark...
Using SMA parameters: short_window=20, long_window=50

Running SMA strategy for AOS...

AOS SMA Strategy Performance Summary:
SMA Strategy Return: 0.5499 (54.99%)
Buy and Hold Return: 8.2428 (824.28%)
SMA Sharpe Ratio: 0.1843
Total Trades: 86

Running SMA strategy for AES...

AES SMA Strategy Performance Summary:
SMA Strategy Return: 0.0013 (0.13%)
Buy and Hold Return: -0.0490 (-4.90%)
SMA Sharpe Ratio: 0.0004
Total Trades: 94

Running SMA strategy for ANET...

ANET SMA Strategy Performance Summary:
SMA Strategy Return: 3.4209 (342.09%)
Buy and Hold Return: 31.8175 (3181.75%)
SMA Sharpe Ratio: 0.9677
Total Trades: 55

Running SMA strategy for ALL...

ALL SMA Strategy Performance Summary:
SMA Strategy Return: 1.5533 (155.33%)
Buy and Hold Return: 5.3650 (536.50%)
SMA Sharpe Ratio: 0.6306
Total Trades: 79


In [19]:
# Debug function to identify return calculation issues
def debug_returns(ticker_signals, initial_investment=10000):
    """Analyze portfolio value progression to identify unrealistic jumps"""
    import pandas as pd
    import numpy as np
    
    # Calculate daily returns and check for excessive values
    ticker_signals = ticker_signals.sort_values('Date')
    
    # Initialize debugging columns
    ticker_signals['Debug_Return'] = 0.0
    ticker_signals['Debug_Value'] = initial_investment
    ticker_signals['Is_Suspicious'] = False
    
    for i in range(1, len(ticker_signals)):
        prev_value = ticker_signals['Debug_Value'].iloc[i-1]
        curr_value = ticker_signals['Portfolio_Value'].iloc[i]
        
        # Calculate daily return
        daily_return = (curr_value / prev_value) - 1
        
        # Flag suspicious returns (e.g., >50% daily gain)
        is_suspicious = abs(daily_return) > 0.5
        
        # Store debug values
        ticker_signals.loc[ticker_signals.index[i], 'Debug_Return'] = daily_return
        ticker_signals.loc[ticker_signals.index[i], 'Debug_Value'] = curr_value
        ticker_signals.loc[ticker_signals.index[i], 'Is_Suspicious'] = is_suspicious
    
    # Find the biggest jumps
    biggest_jumps = ticker_signals.sort_values('Debug_Return', ascending=False).head(5)
    
    print("\n=== Return Analysis ===")
    print(f"Initial portfolio value: ${initial_investment:.2f}")
    print(f"Final portfolio value: ${ticker_signals['Portfolio_Value'].iloc[-1]:.2f}")
    print(f"Total return: {(ticker_signals['Portfolio_Value'].iloc[-1] / initial_investment - 1) * 100:.2f}%")
    print(f"Number of suspicious daily returns (>50%): {ticker_signals['Is_Suspicious'].sum()}")
    
    print("\n=== Biggest Value Jumps ===")
    for _, row in biggest_jumps.iterrows():
        print(f"Date: {row['Date'].strftime('%Y-%m-%d')}, "
              f"Return: {row['Debug_Return']*100:.2f}%, "
              f"Value before: ${row['Debug_Value']:.2f}, " 
              f"Value after: ${row['Portfolio_Value']:.2f}")
    
    return ticker_signals

# Fixed trading strategy with improved error handling for realistic returns
def fixed_weather_strategy(ticker_data, base_model, all_model, base_features, all_features, 
                          key_weather_features, initial_investment=10000):
    """
    Fixed version of the weather-sensitive trading strategy with better return calculations
    """
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import os
    from sklearn.preprocessing import StandardScaler
    
    print(f"\nRunning fixed strategy for {ticker_data['Ticker'].iloc[0]}...")
    
    # Prepare features for base model
    X_base = ticker_data[base_features]
    scaler_base = StandardScaler()
    X_base_scaled = scaler_base.fit_transform(X_base)
    
    # Get base predictions
    base_pred = base_model.predict(X_base_scaled)
    base_prob = base_model.predict_proba(X_base_scaled)[:, 1]
    
    # Prepare features for weather model
    X_all = ticker_data[all_features]
    scaler_all = StandardScaler()
    X_all_scaled = scaler_all.fit_transform(X_all)
    
    # Get weather model predictions
    all_pred = all_model.predict(X_all_scaled)
    all_prob = all_model.predict_proba(X_all_scaled)[:, 1]
    
    # Create signals dataframe
    ticker_signals = pd.DataFrame({
        'Date': ticker_data['Date'],
        'Ticker': ticker_data['Ticker'],
        'Close': ticker_data['Close'],
        'Base_Signal': base_pred,
        'Base_Confidence': base_prob,
        'Weather_Signal': all_pred,
        'Weather_Confidence': all_prob
    })
    
    # Add key weather indicators
    for feature in key_weather_features:
        if feature in ticker_data.columns:
            ticker_signals[feature] = ticker_data[feature]
            
            # Identify extreme values (top and bottom 10%)
            feature_q90 = ticker_data[feature].quantile(0.90)
            feature_q10 = ticker_data[feature].quantile(0.10)
            
            ticker_signals[f'{feature}_extreme_high'] = (ticker_data[feature] > feature_q90).astype(int)
            ticker_signals[f'{feature}_extreme_low'] = (ticker_data[feature] < feature_q10).astype(int)
    
    # Create a weather extremity score
    extreme_cols = [col for col in ticker_signals.columns if 'extreme_high' in col or 'extreme_low' in col]
    if extreme_cols:
        ticker_signals['weather_extremity'] = ticker_signals[extreme_cols].sum(axis=1)
    else:
        ticker_signals['weather_extremity'] = 0
    
    # Add season information if available
    if 'season' in ticker_data.columns:
        ticker_signals['season'] = ticker_data['season']
    
    # Sort by date for chronological simulation
    ticker_signals = ticker_signals.sort_values('Date')
    
    # Create strategy signals
    ticker_signals['Strategy_Signal'] = 'Hold'
    
    # Both models predict buy
    buy_mask = (ticker_signals['Base_Signal'] == 1) & (ticker_signals['Weather_Signal'] == 1)
    ticker_signals.loc[buy_mask, 'Strategy_Signal'] = 'Buy'
    
    # Both models predict sell
    sell_mask = (ticker_signals['Base_Signal'] == 0) & (ticker_signals['Weather_Signal'] == 0)
    ticker_signals.loc[sell_mask, 'Strategy_Signal'] = 'Sell'
    
    # Models disagree but extreme weather
    extreme_mask = ticker_signals['weather_extremity'] >= 1
    
    # Weather says buy, base says sell, extreme weather
    weather_buy_mask = (ticker_signals['Weather_Signal'] == 1) & (ticker_signals['Base_Signal'] == 0) & extreme_mask
    ticker_signals.loc[weather_buy_mask, 'Strategy_Signal'] = 'Buy_Weather'
    
    # Weather says sell, base says buy, extreme weather
    weather_sell_mask = (ticker_signals['Weather_Signal'] == 0) & (ticker_signals['Base_Signal'] == 1) & extreme_mask
    ticker_signals.loc[weather_sell_mask, 'Strategy_Signal'] = 'Sell_Weather'
    
    # Initialize portfolio tracking - WEATHER STRATEGY
    ticker_signals['Position'] = 0  # 0 = no position, 1 = long position
    ticker_signals['Position_Size'] = 0.0  # Percentage of portfolio
    ticker_signals['Portfolio_Value'] = initial_investment  # Initialize all rows
    ticker_signals['Cash'] = initial_investment  # Initialize all rows
    ticker_signals['Shares'] = 0  # Initialize all rows
    ticker_signals['Trade'] = 'None'
    ticker_signals['Trade_Price'] = np.nan
    ticker_signals['Return'] = 0.0
    
    # Initialize portfolio tracking - BASIC STRATEGY  
    ticker_signals['Basic_Position'] = 0
    ticker_signals['Basic_Portfolio'] = initial_investment  # Initialize all rows
    ticker_signals['Basic_Cash'] = initial_investment  # Initialize all rows
    ticker_signals['Basic_Shares'] = 0  # Initialize all rows
    ticker_signals['Basic_Trade'] = 'None'
    ticker_signals['Basic_Return'] = 0.0
    
    # Trading parameters
    transaction_cost_pct = 0.001  # 0.1% per trade
    slippage_pct = 0.001  # 0.1% slippage
    
    # WEATHER STRATEGY SIMULATION
    for i in range(1, len(ticker_signals)):
        prev_idx = ticker_signals.index[i-1]
        curr_idx = ticker_signals.index[i]
        
        # Get previous and current state
        prev_row = ticker_signals.loc[prev_idx]
        curr_row = ticker_signals.loc[curr_idx]
        
        # Default: carry forward previous position and values
        ticker_signals.loc[curr_idx, 'Position'] = prev_row['Position']
        ticker_signals.loc[curr_idx, 'Cash'] = prev_row['Cash']
        ticker_signals.loc[curr_idx, 'Shares'] = prev_row['Shares']
        ticker_signals.loc[curr_idx, 'Position_Size'] = prev_row['Position_Size']
        
        # Calculate current value before any trades (mark-to-market)
        if prev_row['Position'] == 1:
            # Value existing position at current price
            shares_value = prev_row['Shares'] * curr_row['Close']
            curr_portfolio_value = shares_value + prev_row['Cash']
        else:
            # Just cash
            curr_portfolio_value = prev_row['Cash']
        
        # Execute trades based on signals
        if prev_row['Strategy_Signal'] in ['Buy', 'Buy_Weather'] and prev_row['Position'] == 0:
            # Determine position size
            position_size = 1.0 if prev_row['Strategy_Signal'] == 'Buy' else 0.75
            
            # Buy with slippage
            buy_price = prev_row['Close'] * (1 + slippage_pct)
            cash_to_invest = prev_row['Cash'] * position_size
            transaction_cost = cash_to_invest * transaction_cost_pct
            available_cash = cash_to_invest - transaction_cost
            
            # Calculate shares and remaining cash
            shares_to_buy = available_cash / buy_price if buy_price > 0 else 0
            cash_remaining = prev_row['Cash'] - cash_to_invest
            
            # Update position
            ticker_signals.loc[curr_idx, 'Position'] = 1
            ticker_signals.loc[curr_idx, 'Position_Size'] = position_size
            ticker_signals.loc[curr_idx, 'Cash'] = cash_remaining
            ticker_signals.loc[curr_idx, 'Shares'] = shares_to_buy
            ticker_signals.loc[curr_idx, 'Trade'] = 'Buy'
            ticker_signals.loc[curr_idx, 'Trade_Price'] = buy_price
            
            # Calculate new portfolio value after trade
            new_shares_value = shares_to_buy * curr_row['Close']
            ticker_signals.loc[curr_idx, 'Portfolio_Value'] = new_shares_value + cash_remaining
            
        elif prev_row['Strategy_Signal'] in ['Sell', 'Sell_Weather'] and prev_row['Position'] == 1:
            # Sell with slippage
            sell_price = prev_row['Close'] * (1 - slippage_pct)
            shares_value = prev_row['Shares'] * sell_price
            transaction_cost = shares_value * transaction_cost_pct
            cash_after_sale = shares_value - transaction_cost + prev_row['Cash']
            
            # Update position
            ticker_signals.loc[curr_idx, 'Position'] = 0
            ticker_signals.loc[curr_idx, 'Position_Size'] = 0.0
            ticker_signals.loc[curr_idx, 'Cash'] = cash_after_sale
            ticker_signals.loc[curr_idx, 'Shares'] = 0
            ticker_signals.loc[curr_idx, 'Trade'] = 'Sell'
            ticker_signals.loc[curr_idx, 'Trade_Price'] = sell_price
            
            # Portfolio value after selling is just cash
            ticker_signals.loc[curr_idx, 'Portfolio_Value'] = cash_after_sale
        else:
            # No trade, just update portfolio value
            ticker_signals.loc[curr_idx, 'Portfolio_Value'] = curr_portfolio_value
        
        # Calculate daily return - prevent division by zero
        if prev_row['Portfolio_Value'] > 0:
            ticker_signals.loc[curr_idx, 'Return'] = (
                ticker_signals.loc[curr_idx, 'Portfolio_Value'] / 
                prev_row['Portfolio_Value'] - 1
            )
        else:
            ticker_signals.loc[curr_idx, 'Return'] = 0
    
    # BASIC STRATEGY SIMULATION
    for i in range(1, len(ticker_signals)):
        prev_idx = ticker_signals.index[i-1]
        curr_idx = ticker_signals.index[i]
        
        # Get previous and current state
        prev_row = ticker_signals.loc[prev_idx]
        curr_row = ticker_signals.loc[curr_idx]
        
        # Default: carry forward
        ticker_signals.loc[curr_idx, 'Basic_Position'] = prev_row['Basic_Position']
        ticker_signals.loc[curr_idx, 'Basic_Cash'] = prev_row['Basic_Cash']
        ticker_signals.loc[curr_idx, 'Basic_Shares'] = prev_row['Basic_Shares']
        
        # Calculate current value before any trades
        if prev_row['Basic_Position'] == 1:
            # Value existing position at current price
            shares_value = prev_row['Basic_Shares'] * curr_row['Close']
            curr_portfolio_value = shares_value + prev_row['Basic_Cash']
        else:
            # Just cash
            curr_portfolio_value = prev_row['Basic_Cash']
        
        # Execute trades based on base model signals
        basic_signal = 'Buy' if prev_row['Base_Signal'] == 1 else 'Sell'
        
        if basic_signal == 'Buy' and prev_row['Basic_Position'] == 0:
            # Buy with costs
            buy_price = prev_row['Close'] * (1 + slippage_pct)
            transaction_cost = prev_row['Basic_Cash'] * transaction_cost_pct
            available_cash = prev_row['Basic_Cash'] - transaction_cost
            
            # Calculate shares
            shares_to_buy = available_cash / buy_price if buy_price > 0 else 0
            
            # Update position
            ticker_signals.loc[curr_idx, 'Basic_Position'] = 1
            ticker_signals.loc[curr_idx, 'Basic_Cash'] = 0
            ticker_signals.loc[curr_idx, 'Basic_Shares'] = shares_to_buy
            ticker_signals.loc[curr_idx, 'Basic_Trade'] = 'Buy'
            
            # Portfolio value after buying
            ticker_signals.loc[curr_idx, 'Basic_Portfolio'] = shares_to_buy * curr_row['Close']
            
        elif basic_signal == 'Sell' and prev_row['Basic_Position'] == 1:
            # Sell with costs
            sell_price = prev_row['Close'] * (1 - slippage_pct)
            shares_value = prev_row['Basic_Shares'] * sell_price
            transaction_cost = shares_value * transaction_cost_pct
            cash_after_sale = shares_value - transaction_cost
            
            # Update position
            ticker_signals.loc[curr_idx, 'Basic_Position'] = 0
            ticker_signals.loc[curr_idx, 'Basic_Cash'] = cash_after_sale
            ticker_signals.loc[curr_idx, 'Basic_Shares'] = 0
            ticker_signals.loc[curr_idx, 'Basic_Trade'] = 'Sell'
            
            # Portfolio value after selling is just cash
            ticker_signals.loc[curr_idx, 'Basic_Portfolio'] = cash_after_sale
        else:
            # No trade, just update portfolio value
            ticker_signals.loc[curr_idx, 'Basic_Portfolio'] = curr_portfolio_value
        
        # Calculate daily return - prevent division by zero
        if prev_row['Basic_Portfolio'] > 0:
            ticker_signals.loc[curr_idx, 'Basic_Return'] = (
                ticker_signals.loc[curr_idx, 'Basic_Portfolio'] / 
                prev_row['Basic_Portfolio'] - 1
            )
        else:
            ticker_signals.loc[curr_idx, 'Basic_Return'] = a0
    
    # Calculate cumulative returns
    ticker_signals['Cum_Return'] = (1 + ticker_signals['Return']).cumprod()
    ticker_signals['Basic_Cum_Return'] = (1 + ticker_signals['Basic_Return']).cumprod()
    
    # Add buy-and-hold benchmark
    ticker_signals['BuyHold_Return'] = ticker_signals['Close'] / ticker_signals['Close'].iloc[0]
    
    # Run debug analysis
    debug_weather = debug_returns(ticker_signals[['Date', 'Portfolio_Value']], initial_investment)
    debug_basic = debug_returns(ticker_signals[['Date', 'Basic_Portfolio']], initial_investment)
    
    # Calculate performance metrics
    final_weather_return = ticker_signals['Portfolio_Value'].iloc[-1] / initial_investment - 1
    final_basic_return = ticker_signals['Basic_Portfolio'].iloc[-1] / initial_investment - 1
    final_buy_hold = ticker_signals['BuyHold_Return'].iloc[-1] - 1
    
    print(f"\n{ticker_signals['Ticker'].iloc[0]} Performance Summary:")
    print(f"Weather-Sensitive Strategy Return: {final_weather_return:.4f} ({final_weather_return * 100:.2f}%)")
    print(f"Basic Strategy Return: {final_basic_return:.4f} ({final_basic_return * 100:.2f}%)")
    print(f"Buy and Hold Return: {final_buy_hold:.4f} ({final_buy_hold * 100:.2f}%)")
    print(f"Outperformance vs Basic: {(final_weather_return - final_basic_return) * 100:.2f}%")
    
    # Create visualization
    try:
        plt.figure(figsize=(14, 10))
        
        # Portfolio value chart
        plt.subplot(3, 1, 1)
        plt.plot(ticker_signals['Date'], ticker_signals['Portfolio_Value'], label='Weather Strategy')
        plt.plot(ticker_signals['Date'], ticker_signals['Basic_Portfolio'], label='Basic Strategy')
        plt.axhline(y=initial_investment, color='k', linestyle='--', alpha=0.3)
        
        # Mark weather strategy trades
        buys = ticker_signals[ticker_signals['Trade'] == 'Buy']
        sells = ticker_signals[ticker_signals['Trade'] == 'Sell']
        
        if len(buys) > 0:
            plt.scatter(buys['Date'], buys['Portfolio_Value'], 
                        marker='^', color='green', s=100, label='Buy')
        if len(sells) > 0:
            plt.scatter(sells['Date'], sells['Portfolio_Value'], 
                        marker='v', color='red', s=100, label='Sell')
        
        plt.title(f'{ticker_signals["Ticker"].iloc[0]} Portfolio Performance')
        plt.ylabel('Portfolio Value ($)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Weather extremity and trades
        plt.subplot(3, 1, 2)
        plt.bar(ticker_signals['Date'], ticker_signals['weather_extremity'], 
                color='skyblue', alpha=0.6, label='Weather Extremity')
        
        # Mark weather-based trades
        weather_buys = ticker_signals[ticker_signals['Strategy_Signal'] == 'Buy_Weather']
        weather_sells = ticker_signals[ticker_signals['Strategy_Signal'] == 'Sell_Weather']
        
        if len(weather_buys) > 0:
            plt.scatter(weather_buys['Date'], weather_buys['weather_extremity'] + 0.2, 
                       marker='^', color='green', s=80, label='Weather Buy Signal')
        
        if len(weather_sells) > 0:
            plt.scatter(weather_sells['Date'], weather_sells['weather_extremity'] + 0.2, 
                       marker='v', color='red', s=80, label='Weather Sell Signal')
        
        plt.title(f'{ticker_signals["Ticker"].iloc[0]} Weather Extremity and Weather-Based Trades')
        plt.ylabel('Weather Extremity Score')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Cumulative returns comparison
        plt.subplot(3, 1, 3)
        
        plt.plot(ticker_signals['Date'], ticker_signals['Cum_Return'], 
                label='Weather-Sensitive Strategy', color='blue')
        plt.plot(ticker_signals['Date'], ticker_signals['Basic_Cum_Return'], 
                label='Basic Strategy', color='orange', linestyle='--')
        plt.plot(ticker_signals['Date'], ticker_signals['BuyHold_Return'], 
                label='Buy and Hold', color='green', linestyle=':')
        
        plt.title(f'{ticker_signals["Ticker"].iloc[0]} Cumulative Returns Comparison')
        plt.ylabel('Cumulative Return (Starting = 1)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(os.path.join(weather_data_dir, f'{ticker_signals["Ticker"].iloc[0]}_fixed_strategy.png'))
        plt.close()
    except Exception as e:
        print(f"Error creating visualization: {e}")
    
    return ticker_signals

# Main function to run fixed strategy with debugging
def run_fixed_weather_strategy():
    """Run the fixed weather strategy with proper return calculations"""
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import os
    
    print("\n--- Running Fixed Weather-Sensitive Trading Strategy ---")
    
    try:
        # Select tickers to analyze
        if 'seasonal_impact_df' in globals() and len(seasonal_impact_df) > 0:
            weather_sensitive_stocks = seasonal_impact_df.groupby('Ticker')['Weather_Impact'].mean()
            weather_sensitive_stocks = weather_sensitive_stocks.sort_values(ascending=False).head(5).index.tolist()
        else:
            # Fallback: use top 5 stocks by data volume
            weather_sensitive_stocks = combined_df['Ticker'].value_counts().head(5).index.tolist()
        
        print(f"Weather sensitive stocks selected: {weather_sensitive_stocks}")
        
        # Identify key weather features
        key_weather_features = weather_in_top[:min(3, len(weather_in_top))]
        print(f"Using key weather features: {key_weather_features}")
        
        # Run the fixed strategy for each ticker
        strategy_results = []
        performance_summary = []
        
        for ticker in weather_sensitive_stocks:
            # Get ticker data
            ticker_data = combined_df[combined_df['Ticker'] == ticker].copy()
            
            if len(ticker_data) < 50:
                print(f"Not enough data for {ticker}, skipping...")
                continue
            
            # Run fixed strategy
            ticker_results = fixed_weather_strategy(
                ticker_data, base_model, all_model, 
                base_features, all_features, key_weather_features
            )
            
            # Add to results
            strategy_results.append(ticker_results)
            
            # Calculate performance metrics
            final_weather_return = ticker_results['Portfolio_Value'].iloc[-1] / 10000 - 1
            final_basic_return = ticker_results['Basic_Portfolio'].iloc[-1] / 10000 - 1
            final_buy_hold = ticker_results['BuyHold_Return'].iloc[-1] - 1
            
            # Calculate risk metrics
            weather_vol = ticker_results['Return'].std() * np.sqrt(252)  # Annualized
            basic_vol = ticker_results['Basic_Return'].std() * np.sqrt(252)
            
            # Calculate Sharpe ratio (with error handling)
            weather_sharpe = (final_weather_return / len(ticker_results) * 252) / max(weather_vol, 0.0001)
            basic_sharpe = (final_basic_return / len(ticker_results) * 252) / max(basic_vol, 0.0001)
            
            # Count trades
            weather_trades = len(ticker_results[ticker_results['Trade'] != 'None'])
            basic_trades = len(ticker_results[ticker_results['Basic_Trade'] != 'None'])
            
            # Count profitable trades
            profitable_weather = ticker_results[ticker_results['Trade'] == 'Sell']
            profitable_weather = profitable_weather[profitable_weather['Return'] > 0]
            weather_win_rate = len(profitable_weather) / max(1, len(ticker_results[ticker_results['Trade'] == 'Sell']))
            
            # Add to performance summary
            performance_summary.append({
                'Ticker': ticker,
                'Weather_Return': final_weather_return,
                'Basic_Return': final_basic_return,
                'Buy_Hold_Return': final_buy_hold,
                'Outperformance': final_weather_return - final_basic_return,
                'Weather_Sharpe': weather_sharpe,
                'Basic_Sharpe': basic_sharpe,
                'Weather_Trades': weather_trades,
                'Basic_Trades': basic_trades,
                'Weather_Win_Rate': weather_win_rate
            })
        
        # Create summary dataframe
        if performance_summary:
            summary_df = pd.DataFrame(performance_summary)
            
            # Print overall results
            print("\n=== Overall Fixed Weather-Sensitive Strategy Results ===")
            print(f"Average Weather Strategy Return: {summary_df['Weather_Return'].mean():.4f} ({summary_df['Weather_Return'].mean() * 100:.2f}%)")
            print(f"Average Basic Strategy Return: {summary_df['Basic_Return'].mean():.4f} ({summary_df['Basic_Return'].mean() * 100:.2f}%)")
            print(f"Average Buy & Hold Return: {summary_df['Buy_Hold_Return'].mean():.4f} ({summary_df['Buy_Hold_Return'].mean() * 100:.2f}%)")
            print(f"Average Outperformance: {summary_df['Outperformance'].mean():.4f} ({summary_df['Outperformance'].mean() * 100:.2f}%)")
            print(f"Average Weather Sharpe: {summary_df['Weather_Sharpe'].mean():.4f}")
            print(f"Average Basic Sharpe: {summary_df['Basic_Sharpe'].mean():.4f}")
            
            # Create comparison plot
            try:
                plt.figure(figsize=(12, 8))
                
                x = np.arange(len(summary_df))
                width = 0.25
                
                plt.bar(x - width, summary_df['Weather_Return'] * 100, width, label='Weather Strategy')
                plt.bar(x, summary_df['Basic_Return'] * 100, width, label='Basic Strategy')
                plt.bar(x + width, summary_df['Buy_Hold_Return'] * 100, width, label='Buy & Hold')
                
                plt.xlabel('Ticker')
                plt.ylabel('Return (%)')
                plt.title('Strategy Returns by Ticker')
                plt.xticks(x, summary_df['Ticker'])
                plt.legend()
                plt.grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.savefig(os.path.join(weather_data_dir, 'fixed_weather_strategy_comparison.png'))
                plt.close()
                
                # Plot Sharpe ratio comparison
                plt.figure(figsize=(12, 6))
                
                plt.bar(x - width/2, summary_df['Weather_Sharpe'], width, label='Weather Strategy')
                plt.bar(x + width/2, summary_df['Basic_Sharpe'], width, label='Basic Strategy')
                
                plt.xlabel('Ticker')
                plt.ylabel('Sharpe Ratio')
                plt.title('Strategy Sharpe Ratios by Ticker')
                plt.xticks(x, summary_df['Ticker'])
                plt.legend()
                plt.grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.savefig(os.path.join(weather_data_dir, 'fixed_weather_strategy_sharpe.png'))
                plt.close()
            except Exception as e:
                print(f"Error creating summary plots: {e}")
            
            return strategy_results, summary_df
        else:
            print("No performance summary data collected.")
            return None, None
    except Exception as e:
        print(f"Error in fixed weather strategy: {e}")
        return None, None

# Run the fixed strategy
fixed_results, fixed_summary = run_fixed_weather_strategy()


--- Running Fixed Weather-Sensitive Trading Strategy ---
Weather sensitive stocks selected: ['AOS', 'AES', 'ANET', 'ALL', 'APH']
Using key weather features: ['wind_speed_10m_max', 'wind_speed_10m_max_lag2', 'wind_speed_10m_max_lag1']

Running fixed strategy for AOS...

=== Return Analysis ===
Initial portfolio value: $10000.00
Final portfolio value: $978349117.56
Total return: 9783391.18%
Number of suspicious daily returns (>50%): 0

=== Biggest Value Jumps ===
Date: 2023-01-31, Return: 13.44%, Value before: $353791695.63, Value after: $353791695.63
Date: 2011-10-21, Return: 10.34%, Value before: $75585.19, Value after: $75585.19
Date: 2021-10-28, Return: 9.80%, Value before: $82341794.60, Value after: $82341794.60
Date: 2018-10-30, Return: 8.14%, Value before: $8464254.52, Value after: $8464254.52
Date: 2020-05-18, Return: 7.65%, Value before: $26533142.04, Value after: $26533142.04
Error in fixed weather strategy: 'Portfolio_Value'
